In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import array
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import train_test_split 
from sklearn import linear_model, preprocessing 
import warnings
import statsmodels.api as sm
!pip install GEKKO
!pip install ortools

from gekko import GEKKO

In [4]:
Download=pd.read_csv("DKSalaries-5.csv")

In [7]:
Download

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,QB,Tom Brady (19640018),Tom Brady,19640018,CPT,18600,TB@PHI 10/14/2021 08:20PM ET,TB,29.26
1,QB,Jalen Hurts (19640019),Jalen Hurts,19640019,CPT,17700,TB@PHI 10/14/2021 08:20PM ET,PHI,26.24
2,WR,Mike Evans (19640020),Mike Evans,19640020,CPT,15300,TB@PHI 10/14/2021 08:20PM ET,TB,19.66
3,WR,Chris Godwin (19640021),Chris Godwin,19640021,CPT,14100,TB@PHI 10/14/2021 08:20PM ET,TB,17.16
4,WR,DeVonta Smith (19640022),DeVonta Smith,19640022,CPT,13200,TB@PHI 10/14/2021 08:20PM ET,PHI,13.28
...,...,...,...,...,...,...,...,...,...
97,TE,Jack Stoll (19640115),Jack Stoll,19640115,FLEX,200,TB@PHI 10/14/2021 08:20PM ET,PHI,0.00
98,WR,Scotty Miller (19640116),Scotty Miller,19640116,FLEX,200,TB@PHI 10/14/2021 08:20PM ET,TB,1.03
99,WR,Justin Watson (19640117),Justin Watson,19640117,FLEX,200,TB@PHI 10/14/2021 08:20PM ET,TB,0.00
100,TE,Jason Croom (19640118),Jason Croom,19640118,FLEX,200,TB@PHI 10/14/2021 08:20PM ET,PHI,0.00


In [11]:
Download.to_csv("TBVsPHI.csv")

In [12]:
Download["KingCost"]=Download["Salary"]*1.5

<ipython-input-12-e8e230d3b3cc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Download["KingCost"]=Download["Salary"]*1.5


In [13]:
Download=Download.loc[((Download['Roster Position'] != "CPT"))]

In [14]:
Download.index = np.arange(1, len(Download)+1)
Download

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,KingCost
1,QB,Tom Brady (19640069),Tom Brady,19640069,FLEX,12400,TB@PHI 10/14/2021 08:20PM ET,TB,29.26,18600.0
2,QB,Jalen Hurts (19640070),Jalen Hurts,19640070,FLEX,11800,TB@PHI 10/14/2021 08:20PM ET,PHI,26.24,17700.0
3,WR,Mike Evans (19640071),Mike Evans,19640071,FLEX,10200,TB@PHI 10/14/2021 08:20PM ET,TB,19.66,15300.0
4,WR,Chris Godwin (19640072),Chris Godwin,19640072,FLEX,9400,TB@PHI 10/14/2021 08:20PM ET,TB,17.16,14100.0
5,WR,DeVonta Smith (19640073),DeVonta Smith,19640073,FLEX,8800,TB@PHI 10/14/2021 08:20PM ET,PHI,13.28,13200.0
6,WR,Antonio Brown (19640074),Antonio Brown,19640074,FLEX,8200,TB@PHI 10/14/2021 08:20PM ET,TB,19.28,12300.0
7,RB,Leonard Fournette (19640075),Leonard Fournette,19640075,FLEX,7800,TB@PHI 10/14/2021 08:20PM ET,TB,13.36,11700.0
8,TE,Rob Gronkowski (19640076),Rob Gronkowski,19640076,FLEX,7000,TB@PHI 10/14/2021 08:20PM ET,TB,19.47,10500.0
9,RB,Miles Sanders (19640077),Miles Sanders,19640077,FLEX,6600,TB@PHI 10/14/2021 08:20PM ET,PHI,10.10,9900.0
10,QB,Blaine Gabbert (19640078),Blaine Gabbert,19640078,FLEX,6000,TB@PHI 10/14/2021 08:20PM ET,TB,0.62,9000.0


In [15]:
Modified=Download

8, 10,11, 12, 13, 14, 21, 
30 and under
46, 47, 48, 49, 50, 51

In [16]:
Modified = Modified.drop(8)
Modified = Modified.drop(10)
Modified = Modified.drop(11)
Modified = Modified.drop(12)
Modified = Modified.drop(13)
Modified = Modified.drop(14)
Modified = Modified.drop(21)

Modified = Modified.drop(49)
Modified = Modified.drop(48)
Modified = Modified.drop(47)
Modified = Modified.drop(46)
Modified = Modified.drop(45)
Modified = Modified.drop(44)
Modified = Modified.drop(43)
Modified = Modified.drop(42)
Modified = Modified.drop(41)
Modified = Modified.drop(40)
Modified = Modified.drop(39)
Modified = Modified.drop(38)
Modified = Modified.drop(37)
Modified = Modified.drop(36)
Modified = Modified.drop(35)
Modified = Modified.drop(34)
Modified = Modified.drop(33)
Modified = Modified.drop(32)
Modified = Modified.drop(31)
Modified = Modified.drop(30)


Modified = Modified.drop(51)
Modified = Modified.drop(50)

In [26]:
Modified

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,KingCost
1,QB,Tom Brady (19640069),Tom Brady,19640069,FLEX,12400,TB@PHI 10/14/2021 08:20PM ET,TB,29.26,18600.0
2,QB,Jalen Hurts (19640070),Jalen Hurts,19640070,FLEX,11800,TB@PHI 10/14/2021 08:20PM ET,PHI,26.24,17700.0
3,WR,Mike Evans (19640071),Mike Evans,19640071,FLEX,10200,TB@PHI 10/14/2021 08:20PM ET,TB,19.66,15300.0
4,WR,Chris Godwin (19640072),Chris Godwin,19640072,FLEX,9400,TB@PHI 10/14/2021 08:20PM ET,TB,17.16,14100.0
5,WR,DeVonta Smith (19640073),DeVonta Smith,19640073,FLEX,8800,TB@PHI 10/14/2021 08:20PM ET,PHI,13.28,13200.0
6,WR,Antonio Brown (19640074),Antonio Brown,19640074,FLEX,8200,TB@PHI 10/14/2021 08:20PM ET,TB,19.28,12300.0
7,RB,Leonard Fournette (19640075),Leonard Fournette,19640075,FLEX,7800,TB@PHI 10/14/2021 08:20PM ET,TB,13.36,11700.0
8,RB,Miles Sanders (19640077),Miles Sanders,19640077,FLEX,6600,TB@PHI 10/14/2021 08:20PM ET,PHI,10.10,9900.0
9,RB,Giovani Bernard (19640083),Giovani Bernard,19640083,FLEX,5200,TB@PHI 10/14/2021 08:20PM ET,TB,9.60,7800.0
10,WR,Jalen Reagor (19640084),Jalen Reagor,19640084,FLEX,5000,TB@PHI 10/14/2021 08:20PM ET,PHI,7.64,7500.0


In [27]:

Modified.index = np.arange(1, len(Modified)+1)

In [28]:
Modified

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,KingCost
1,QB,Tom Brady (19640069),Tom Brady,19640069,FLEX,12400,TB@PHI 10/14/2021 08:20PM ET,TB,29.26,18600.0
2,QB,Jalen Hurts (19640070),Jalen Hurts,19640070,FLEX,11800,TB@PHI 10/14/2021 08:20PM ET,PHI,26.24,17700.0
3,WR,Mike Evans (19640071),Mike Evans,19640071,FLEX,10200,TB@PHI 10/14/2021 08:20PM ET,TB,19.66,15300.0
4,WR,Chris Godwin (19640072),Chris Godwin,19640072,FLEX,9400,TB@PHI 10/14/2021 08:20PM ET,TB,17.16,14100.0
5,WR,DeVonta Smith (19640073),DeVonta Smith,19640073,FLEX,8800,TB@PHI 10/14/2021 08:20PM ET,PHI,13.28,13200.0
6,WR,Antonio Brown (19640074),Antonio Brown,19640074,FLEX,8200,TB@PHI 10/14/2021 08:20PM ET,TB,19.28,12300.0
7,RB,Leonard Fournette (19640075),Leonard Fournette,19640075,FLEX,7800,TB@PHI 10/14/2021 08:20PM ET,TB,13.36,11700.0
8,RB,Miles Sanders (19640077),Miles Sanders,19640077,FLEX,6600,TB@PHI 10/14/2021 08:20PM ET,PHI,10.10,9900.0
9,RB,Giovani Bernard (19640083),Giovani Bernard,19640083,FLEX,5200,TB@PHI 10/14/2021 08:20PM ET,TB,9.60,7800.0
10,WR,Jalen Reagor (19640084),Jalen Reagor,19640084,FLEX,5000,TB@PHI 10/14/2021 08:20PM ET,PHI,7.64,7500.0


In [31]:
Code_list=['x1x2','x3x4','x5x6','x7x8','x9x10',
           'x11x12','x13x14','x15x16','x17x18',
           'x19x20','x21x22','x23x24','x25x26',
           'x27x28','x29x30','x31x32','x33x34',
           'x35x36','x37x38','x39x40', 'x41x42','x43x44']

In [32]:
Modified.index = Code_list

In [33]:
Modified

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,KingCost
x1x2,QB,Tom Brady (19640069),Tom Brady,19640069,FLEX,12400,TB@PHI 10/14/2021 08:20PM ET,TB,29.26,18600.0
x3x4,QB,Jalen Hurts (19640070),Jalen Hurts,19640070,FLEX,11800,TB@PHI 10/14/2021 08:20PM ET,PHI,26.24,17700.0
x5x6,WR,Mike Evans (19640071),Mike Evans,19640071,FLEX,10200,TB@PHI 10/14/2021 08:20PM ET,TB,19.66,15300.0
x7x8,WR,Chris Godwin (19640072),Chris Godwin,19640072,FLEX,9400,TB@PHI 10/14/2021 08:20PM ET,TB,17.16,14100.0
x9x10,WR,DeVonta Smith (19640073),DeVonta Smith,19640073,FLEX,8800,TB@PHI 10/14/2021 08:20PM ET,PHI,13.28,13200.0
x11x12,WR,Antonio Brown (19640074),Antonio Brown,19640074,FLEX,8200,TB@PHI 10/14/2021 08:20PM ET,TB,19.28,12300.0
x13x14,RB,Leonard Fournette (19640075),Leonard Fournette,19640075,FLEX,7800,TB@PHI 10/14/2021 08:20PM ET,TB,13.36,11700.0
x15x16,RB,Miles Sanders (19640077),Miles Sanders,19640077,FLEX,6600,TB@PHI 10/14/2021 08:20PM ET,PHI,10.10,9900.0
x17x18,RB,Giovani Bernard (19640083),Giovani Bernard,19640083,FLEX,5200,TB@PHI 10/14/2021 08:20PM ET,TB,9.60,7800.0
x19x20,WR,Jalen Reagor (19640084),Jalen Reagor,19640084,FLEX,5000,TB@PHI 10/14/2021 08:20PM ET,PHI,7.64,7500.0


In [34]:
Download=Modified

In [35]:
Download2=pd.read_csv("Projections.csv")

In [36]:
Download2.to_csv("Week5 Projections.csv")

In [37]:
Download2['Name']=Download2['playerName']

In [38]:
Download2

,fantasyPointsRank,playerName,teamName,position,byeWeek,games,fantasyPoints,passComp,passAtt,passYds,...,idpTacklesAssist,idpSacks,idpTacklesForLoss,idpPassDefended,idpInt,idpFumblesForced,idpFumblesRecovered,idpSafeties,idpTd,Name
0,1,Christian McCaffrey,CAR,rb,13,MIN,21.5,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,Christian McCaffrey
1,2,Najee Harris,PIT,rb,7,SEA,18.8,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,Najee Harris
2,3,Tyreek Hill,KC,wr,12,@WAS,21.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,Tyreek Hill
3,4,Derrick Henry,TEN,rb,13,BUF,17.5,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,Derrick Henry
4,5,Aaron Jones,GB,rb,13,@CHI,17.5,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,Aaron Jones
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,450,Texans DST,HST,dst,10,@IND,5.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,Texans DST
450,451,Lions DST,DET,dst,9,CIN,5.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,Lions DST
451,452,Bears DST,CHI,dst,10,GB,4.4,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,Bears DST
452,453,Football Team DST,WAS,dst,9,KC,4.1,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,Football Team DST


In [39]:
result = pd.merge(Download,
                 Download2,
                 on='Name', 
                 how='inner')

In [40]:
result

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,KingCost,...,idpTacklesSolo,idpTacklesAssist,idpSacks,idpTacklesForLoss,idpPassDefended,idpInt,idpFumblesForced,idpFumblesRecovered,idpSafeties,idpTd
0,QB,Tom Brady (19640069),Tom Brady,19640069,FLEX,12400,TB@PHI 10/14/2021 08:20PM ET,TB,29.26,18600.0,...,0,0,0,0,0,0,0,0,0,0
1,QB,Jalen Hurts (19640070),Jalen Hurts,19640070,FLEX,11800,TB@PHI 10/14/2021 08:20PM ET,PHI,26.24,17700.0,...,0,0,0,0,0,0,0,0,0,0
2,WR,Mike Evans (19640071),Mike Evans,19640071,FLEX,10200,TB@PHI 10/14/2021 08:20PM ET,TB,19.66,15300.0,...,0,0,0,0,0,0,0,0,0,0
3,WR,Chris Godwin (19640072),Chris Godwin,19640072,FLEX,9400,TB@PHI 10/14/2021 08:20PM ET,TB,17.16,14100.0,...,0,0,0,0,0,0,0,0,0,0
4,WR,DeVonta Smith (19640073),DeVonta Smith,19640073,FLEX,8800,TB@PHI 10/14/2021 08:20PM ET,PHI,13.28,13200.0,...,0,0,0,0,0,0,0,0,0,0
5,WR,Antonio Brown (19640074),Antonio Brown,19640074,FLEX,8200,TB@PHI 10/14/2021 08:20PM ET,TB,19.28,12300.0,...,0,0,0,0,0,0,0,0,0,0
6,RB,Leonard Fournette (19640075),Leonard Fournette,19640075,FLEX,7800,TB@PHI 10/14/2021 08:20PM ET,TB,13.36,11700.0,...,0,0,0,0,0,0,0,0,0,0
7,RB,Miles Sanders (19640077),Miles Sanders,19640077,FLEX,6600,TB@PHI 10/14/2021 08:20PM ET,PHI,10.10,9900.0,...,0,0,0,0,0,0,0,0,0,0
8,RB,Giovani Bernard (19640083),Giovani Bernard,19640083,FLEX,5200,TB@PHI 10/14/2021 08:20PM ET,TB,9.60,7800.0,...,0,0,0,0,0,0,0,0,0,0
9,WR,Jalen Reagor (19640084),Jalen Reagor,19640084,FLEX,5000,TB@PHI 10/14/2021 08:20PM ET,PHI,7.64,7500.0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
Modified=result

In [54]:
Modified= Modified.drop(['AvgPointsPerGame'], axis = 1)
Modified= Modified.drop(['Name + ID'], axis = 1)
Modified= Modified.drop(['ID'], axis = 1)
Modified= Modified.drop(['Game Info'], axis = 1)
Modified= Modified.drop(['byeWeek'], axis = 1)
Modified= Modified.drop(['games'], axis = 1)
Modified= Modified.drop(['position'], axis = 1)
Modified= Modified.drop(['TeamAbbrev'], axis = 1)
Modified= Modified.drop(['Roster Position'], axis = 1)
Modified= Modified.drop(['idpTd'], axis = 1)
Modified= Modified.drop(['idpSafeties'], axis = 1)

KeyError: "['Name + ID'] not found in axis"

In [55]:
Modified.index = np.arange(1, len(Modified)+1)

In [56]:
Modified

,Position,Name,Salary,KingCost,fantasyPointsRank,playerName,teamName,fantasyPoints,passComp,passAtt,...,dstPts35plus,idpTacklesSolo,idpTacklesAssist,idpSacks,idpTacklesForLoss,idpPassDefended,idpInt,idpFumblesForced,idpFumblesRecovered,ratio
1,QB,Tom Brady,12400,18600.0,49,Tom Brady,TB,23.4,27.0899,39.2897,...,0.0,0,0,0,0,0,0,0,0,0.001258
2,QB,Jalen Hurts,11800,17700.0,114,Jalen Hurts,PHI,18.9,21.5978,34.3018,...,0.0,0,0,0,0,0,0,0,0,0.001068
3,WR,Mike Evans,10200,15300.0,26,Mike Evans,TB,17.0,0.0000,0.0000,...,0.0,0,0,0,0,0,0,0,0,0.001111
4,WR,Chris Godwin,9400,14100.0,43,Chris Godwin,TB,15.3,0.0000,0.0000,...,0.0,0,0,0,0,0,0,0,0,0.001085
5,WR,DeVonta Smith,8800,13200.0,71,DeVonta Smith,PHI,12.9,0.0000,0.0000,...,0.0,0,0,0,0,0,0,0,0,0.000977
6,WR,Antonio Brown,8200,12300.0,22,Antonio Brown,TB,17.5,0.0000,0.0000,...,0.0,0,0,0,0,0,0,0,0,0.001423
7,RB,Leonard Fournette,7800,11700.0,16,Leonard Fournette,TB,15.1,0.0000,0.0000,...,0.0,0,0,0,0,0,0,0,0,0.001291
8,RB,Miles Sanders,6600,9900.0,37,Miles Sanders,PHI,12.5,0.0000,0.0000,...,0.0,0,0,0,0,0,0,0,0,0.001263
9,RB,Giovani Bernard,5200,7800.0,144,Giovani Bernard,TB,5.6,0.0000,0.0000,...,0.0,0,0,0,0,0,0,0,0,0.000718
10,WR,Jalen Reagor,5000,7500.0,148,Jalen Reagor,PHI,8.6,0.0000,0.0000,...,0.0,0,0,0,0,0,0,0,0,0.001147


In [57]:
Modified['AvgPointsPerGame']=Modified['fantasyPoints']

In [58]:
Modified['ratio']=Modified['AvgPointsPerGame']/Modified['KingCost']

In [59]:
Code_list=['x1x2','x3x4','x5x6','x7x8','x9x10',
           'x11x12','x13x14','x15x16','x17x18',
           'x19x20','x21x22','x23x24','x25x26',
           'x27x28','x29x30','x31x32','x33x34',
           'x35x36', 'x37x38']

In [60]:
Modified.index = Code_list

In [61]:
Modified

,Position,Name,Salary,KingCost,fantasyPointsRank,playerName,teamName,fantasyPoints,passComp,passAtt,...,idpTacklesSolo,idpTacklesAssist,idpSacks,idpTacklesForLoss,idpPassDefended,idpInt,idpFumblesForced,idpFumblesRecovered,ratio,AvgPointsPerGame
x1x2,QB,Tom Brady,12400,18600.0,49,Tom Brady,TB,23.4,27.0899,39.2897,...,0,0,0,0,0,0,0,0,0.001258,23.4
x3x4,QB,Jalen Hurts,11800,17700.0,114,Jalen Hurts,PHI,18.9,21.5978,34.3018,...,0,0,0,0,0,0,0,0,0.001068,18.9
x5x6,WR,Mike Evans,10200,15300.0,26,Mike Evans,TB,17.0,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0.001111,17.0
x7x8,WR,Chris Godwin,9400,14100.0,43,Chris Godwin,TB,15.3,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0.001085,15.3
x9x10,WR,DeVonta Smith,8800,13200.0,71,DeVonta Smith,PHI,12.9,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0.000977,12.9
x11x12,WR,Antonio Brown,8200,12300.0,22,Antonio Brown,TB,17.5,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0.001423,17.5
x13x14,RB,Leonard Fournette,7800,11700.0,16,Leonard Fournette,TB,15.1,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0.001291,15.1
x15x16,RB,Miles Sanders,6600,9900.0,37,Miles Sanders,PHI,12.5,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0.001263,12.5
x17x18,RB,Giovani Bernard,5200,7800.0,144,Giovani Bernard,TB,5.6,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0.000718,5.6
x19x20,WR,Jalen Reagor,5000,7500.0,148,Jalen Reagor,PHI,8.6,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0.001147,8.6


In [62]:
Modified.sort_values(by='ratio', ascending=False)

,Position,Name,Salary,KingCost,fantasyPointsRank,playerName,teamName,fantasyPoints,passComp,passAtt,...,idpTacklesSolo,idpTacklesAssist,idpSacks,idpTacklesForLoss,idpPassDefended,idpInt,idpFumblesForced,idpFumblesRecovered,ratio,AvgPointsPerGame
x35x36,WR,Tyler Johnson,1000,1500.0,216,Tyler Johnson,TB,5.3,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0.003533,5.3
x31x32,RB,Kenneth Gainwell,2400,3600.0,104,Kenneth Gainwell,PHI,7.6,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0.002111,7.6
x33x34,TE,O.J. Howard,1400,2100.0,196,O.J. Howard,TB,4.0,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0.001905,4.0
x37x38,WR,Greg Ward,800,1200.0,324,Greg Ward,PHI,2.1,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0.001750,2.1
x29x30,TE,Zach Ertz,3200,4800.0,109,Zach Ertz,PHI,8.1,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0.001687,8.1
x11x12,WR,Antonio Brown,8200,12300.0,22,Antonio Brown,TB,17.5,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0.001423,17.5
x13x14,RB,Leonard Fournette,7800,11700.0,16,Leonard Fournette,TB,15.1,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0.001291,15.1
x27x28,K,Ryan Succop,4200,6300.0,425,Ryan Succop,TB,8.0,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0.001270,8.0
x15x16,RB,Miles Sanders,6600,9900.0,37,Miles Sanders,PHI,12.5,0.0000,0.0000,...,0,0,0,0,0,0,0,0,0.001263,12.5
x1x2,QB,Tom Brady,12400,18600.0,49,Tom Brady,TB,23.4,27.0899,39.2897,...,0,0,0,0,0,0,0,0,0.001258,23.4


In [64]:
Download=Modified

In [116]:
m = GEKKO() # create GEKKO model
# create binary variables
x1 = m.Var(integer=True,lb=0,ub=1) 
x2 = m.Var(integer=True,lb=0,ub=1)
x3 = m.Var(integer=True,lb=0,ub=1)
x4 = m.Var(integer=True,lb=0,ub=1)
x5 = m.Var(integer=True,lb=0,ub=1)
x6 = m.Var(integer=True,lb=0,ub=1)
x7 = m.Var(integer=True,lb=0,ub=1)
x8 = m.Var(integer=True,lb=0,ub=1)
x9 = m.Var(integer=True,lb=0,ub=1)
x10 = m.Var(integer=True,lb=0,ub=1)
x11 = m.Var(integer=True,lb=0,ub=1)
x12 = m.Var(integer=True,lb=0,ub=1)
x13 = m.Var(integer=True,lb=0,ub=1)
x14 = m.Var(integer=True,lb=0,ub=1)
x15 = m.Var(integer=True,lb=0,ub=1)
x16 = m.Var(integer=True,lb=0,ub=1)
x17 = m.Var(integer=True,lb=0,ub=1)
x18 = m.Var(integer=True,lb=0,ub=1)
x19 = m.Var(integer=True,lb=0,ub=1)
x20 = m.Var(integer=True,lb=0,ub=1)
x21 = m.Var(integer=True,lb=0,ub=1)
x22 = m.Var(integer=True,lb=0,ub=1)
x23 = m.Var(integer=True,lb=0,ub=1)
x24 = m.Var(integer=True,lb=0,ub=1)
x25 = m.Var(integer=True,lb=0,ub=1)
x26 = m.Var(integer=True,lb=0,ub=1)
x27 = m.Var(integer=True,lb=0,ub=1)
x28 = m.Var(integer=True,lb=0,ub=1)
x29 = m.Var(integer=True,lb=0,ub=1) 
x30 = m.Var(integer=True,lb=0,ub=1)
x31 = m.Var(integer=True,lb=0,ub=1)
x32 = m.Var(integer=True,lb=0,ub=1)
x33 = m.Var(integer=True,lb=0,ub=1)
x34 = m.Var(integer=True,lb=0,ub=1)
x35 = m.Var(integer=True,lb=0,ub=1)
x36 = m.Var(integer=True,lb=0,ub=1)
x37 = m.Var(integer=True,lb=0,ub=1)
x38 = m.Var(integer=True,lb=0,ub=1)




m.Equation(x25+x26+x27+x28==0)

m.Equation(x2+x4+x6+x8+x10+x12+x14+x16+x18+x20+x22+x24+x26+x28+
           x30+x32+x34+x36+x38==1)

m.Equation(x1+x2+x3+x4+x5+x6+x7+x8+x9+x10+x11+x12+x13+x14+
           x15+x16+x17+x18+x19+x20+x21+x22+x23+x24+x25+x26+x27+x28+
           x29+x30+x31+x32+x33+x34+x35+x36+x37+x38== 6)

m.Equation(x1+x2<=1)
m.Equation(x3+x4<=1)
m.Equation(x5+x6<=1)
m.Equation(x7+x8<=1)
m.Equation(x9+x10<=1)
m.Equation(x11+x12<=1)
m.Equation(x13+x14<=1)
m.Equation(x15+x16<=1)
m.Equation(x17+x18<=1)
m.Equation(x19+x20<=1)
m.Equation(x21+x22<=1)
m.Equation(x23+x24<=1)
m.Equation(x25+x26<=1)
m.Equation(x27+x28<=1)
m.Equation(x29+x30<=1)
m.Equation(x31+x32<=1)
m.Equation(x33+x34<=1)
m.Equation(x35+x36<=1)
m.Equation(x37+x38<=1)


m.Equation((x1*Download.iloc[0]['Salary'])+(x2*Download.iloc[0]['Salary']*1.5)+
            (x3*Download.iloc[1]['Salary'])+(x4*Download.iloc[1]['Salary']*1.5)+
            (x5*Download.iloc[2]['Salary'])+(x6*Download.iloc[2]['Salary']*1.5)+
            (x7*Download.iloc[3]['Salary'])+(x8*Download.iloc[3]['Salary']*1.5)+
            (x9*Download.iloc[4]['Salary'])+(x10*Download.iloc[4]['Salary']*1.5)+
            (x11*Download.iloc[5]['Salary'])+(x12*Download.iloc[5]['Salary']*1.5)+
            (x13*Download.iloc[6]['Salary'])+(x14*Download.iloc[6]['Salary']*1.5)+
            (x15*Download.iloc[7]['Salary'])+(x16*Download.iloc[7]['Salary']*1.5)+
            (x17*Download.iloc[8]['Salary'])+(x18*Download.iloc[8]['Salary']*1.5)+
            (x19*Download.iloc[9]['Salary'])+(x20*Download.iloc[9]['Salary']*1.5)+
            (x21*Download.iloc[10]['Salary'])+(x22*Download.iloc[10]['Salary']*1.5)+
            (x23*Download.iloc[11]['Salary'])+(x24*Download.iloc[11]['Salary']*1.5)+
            (x25*Download.iloc[12]['Salary'])+(x26*Download.iloc[12]['Salary']*1.5)+
            (x27*Download.iloc[13]['Salary'])+(x28*Download.iloc[13]['Salary']*1.5)+
            (x29*Download.iloc[14]['Salary'])+(x30*Download.iloc[14]['Salary']*1.5)+
            (x31*Download.iloc[15]['Salary'])+(x32*Download.iloc[15]['Salary']*1.5)+
            (x33*Download.iloc[16]['Salary'])+(x34*Download.iloc[16]['Salary']*1.5)+
            (x35*Download.iloc[17]['Salary'])+(x36*Download.iloc[17]['Salary']*1.5)+
            (x37*Download.iloc[18]['Salary'])+(x38*Download.iloc[18]['Salary']*1.5)<=50000)


m.Maximize(((x1*Download.iloc[0]['AvgPointsPerGame'])+(x2*Download.iloc[0]['AvgPointsPerGame']*1.5)+
        (x3*Download.iloc[1]['AvgPointsPerGame'])+(x4*Download.iloc[1]['AvgPointsPerGame']*1.5)+
        (x5*Download.iloc[2]['AvgPointsPerGame'])+(x6*Download.iloc[2]['AvgPointsPerGame']*1.5)+
        (x7*Download.iloc[3]['AvgPointsPerGame'])+(x8*Download.iloc[3]['AvgPointsPerGame']*1.5)+
        (x9*Download.iloc[4]['AvgPointsPerGame'])+(x10*Download.iloc[4]['AvgPointsPerGame']*1.5)+
        (x11*Download.iloc[5]['AvgPointsPerGame'])+(x12*Download.iloc[5]['AvgPointsPerGame']*1.5)+
        (x13*Download.iloc[6]['AvgPointsPerGame'])+(x14*Download.iloc[6]['AvgPointsPerGame']*1.5)+
        (x15*Download.iloc[7]['AvgPointsPerGame'])+(x16*Download.iloc[7]['AvgPointsPerGame']*1.5)+
        (x17*Download.iloc[8]['AvgPointsPerGame'])+(x18*Download.iloc[8]['AvgPointsPerGame']*1.5)+
        (x19*Download.iloc[9]['AvgPointsPerGame'])+(x20*Download.iloc[9]['AvgPointsPerGame']*1.5)+
        (x21*Download.iloc[10]['AvgPointsPerGame'])+(x22*Download.iloc[10]['AvgPointsPerGame']*1.5)+
        (x23*Download.iloc[11]['AvgPointsPerGame'])+(x24*Download.iloc[11]['AvgPointsPerGame']*1.5)+
        (x25*Download.iloc[12]['AvgPointsPerGame'])+(x26*Download.iloc[12]['AvgPointsPerGame']*1.5)+
        (x27*Download.iloc[13]['AvgPointsPerGame'])+(x28*Download.iloc[13]['AvgPointsPerGame']*1.5)+
        (x29*Download.iloc[14]['AvgPointsPerGame'])+(x30*Download.iloc[14]['AvgPointsPerGame']*1.5)+
        (x31*Download.iloc[15]['AvgPointsPerGame'])+(x32*Download.iloc[15]['AvgPointsPerGame']*1.5)+
        (x33*Download.iloc[16]['AvgPointsPerGame'])+(x34*Download.iloc[16]['AvgPointsPerGame']*1.5)+
        (x35*Download.iloc[17]['AvgPointsPerGame'])+(x36*Download.iloc[17]['AvgPointsPerGame']*1.5)+
        (x37*Download.iloc[18]['AvgPointsPerGame'])+(x38*Download.iloc[18]['AvgPointsPerGame']*1.5)))

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model6 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           58
   Intermediates:            0
   Connections  :            0
   Equations    :           24
   Residuals    :           24
 
 Number of state variables:             58
 Number of total equations: -           23
 Number of slack variables: -           20
 ---------------------------------------
 Degrees of freedom       :             15
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    3 Dpth:    0 Lvs:    3 Obj: -1.00E+02 Gap:       NaN
--Integer Solution:  -9.59E+01 Lowes

In [67]:
m.Equation(x2+x3+x11+x13+x31+x35<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')


apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           59
   Intermediates:            0
   Connections  :            0
   Equations    :           25
   Residuals    :           25
 
 Number of state variables:             59
 Number of total equations: -           24
 Number of slack variables: -           21
 ---------------------------------------
 Degrees of freedom       :             14
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -1.00E+02 Gap:       NaN
--Integer Solution:  -9.59E+01 Lowes

111

In [68]:
m.Equation(x1+x3+x12+x13+x29+x31<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           60
   Intermediates:            0
   Connections  :            0
   Equations    :           26
   Residuals    :           26
 
 Number of state variables:             60
 Number of total equations: -           25
 Number of slack variables: -           22
 ---------------------------------------
 Degrees of freedom       :             13
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -1.00E+02 Gap:       NaN
--Integer Solution:  -9.59E+01 Lowes

x1: 0.0 Tom Brady
x2: 1.0 Tom Brady /King
x3: 0.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 0.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 1.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 0.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 1.0 Antonio Brown
x12: 0.0 Antonio Brown /King
x13: 1.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 0.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 0.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 0.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 0.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 1.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x31: 1.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 0.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 0.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


In [69]:
m.Equation(x2+x7+x11+x13+x29+x31<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           61
   Intermediates:            0
   Connections  :            0
   Equations    :           27
   Residuals    :           27
 
 Number of state variables:             61
 Number of total equations: -           26
 Number of slack variables: -           23
 ---------------------------------------
 Degrees of freedom       :             12
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -1.00E+02 Gap:       NaN
--Integer Solution:  -9.59E+01 Lowes

x1: 0.0 Tom Brady
x2: 1.0 Tom Brady /King
x3: 0.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 1.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 0.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 0.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 1.0 Antonio Brown
x12: 0.0 Antonio Brown /King
x13: 1.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 0.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 0.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 0.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 0.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 1.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x31: 0.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 0.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 1.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


In [70]:
m.Equation(x2+x5+x11+x13+x29+x35<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           62
   Intermediates:            0
   Connections  :            0
   Equations    :           28
   Residuals    :           28
 
 Number of state variables:             62
 Number of total equations: -           27
 Number of slack variables: -           24
 ---------------------------------------
 Degrees of freedom       :             11
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -1.00E+02 Gap:       NaN
--Integer Solution:  -9.59E+01 Lowes

Iter:   159 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:   91 Obj: -9.80E+01 Gap:  1.13E-02
--Integer Solution:  -9.78E+01 Lowest Leaf:  -9.89E+01 Gap:   1.12E-02
Iter:   160 I:  0 Tm:      0.00 NLPi:    3 Dpth:   11 Lvs:   90 Obj: -8.70E+01 Gap:  1.12E-02
Iter:   161 I:  0 Tm:      0.00 NLPi:    5 Dpth:   11 Lvs:   91 Obj: -9.89E+01 Gap:  1.12E-02
--Integer Solution:  -9.78E+01 Lowest Leaf:  -9.89E+01 Gap:   1.12E-02
Iter:   162 I:  0 Tm:      0.00 NLPi:    4 Dpth:   12 Lvs:   90 Obj: -6.83E+01 Gap:  1.12E-02
Iter:   163 I:  0 Tm:      0.00 NLPi:    4 Dpth:   12 Lvs:   91 Obj: -9.89E+01 Gap:  1.12E-02
Iter:   164 I:  0 Tm:      0.00 NLPi:    3 Dpth:   13 Lvs:   92 Obj: -9.89E+01 Gap:  1.12E-02
Iter:   165 I:  0 Tm:      0.00 NLPi:    3 Dpth:   13 Lvs:   94 Obj: -9.80E+01 Gap:  1.12E-02
Iter:   166 I: -1 Tm:      0.00 NLPi:    1 Dpth:    6 Lvs:   93 Obj: -9.89E+01 Gap:  1.12E-02
Iter:   167 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:   95 Obj: -9.81E+01 Gap:  1.12E-02
Iter:   168 

In [71]:
m.Equation(x2+x5+x11+x15+x29+x31<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           63
   Intermediates:            0
   Connections  :            0
   Equations    :           29
   Residuals    :           29
 
 Number of state variables:             63
 Number of total equations: -           28
 Number of slack variables: -           25
 ---------------------------------------
 Degrees of freedom       :             10
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -1.00E+02 Gap:       NaN
--Integer Solution:  -9.59E+01 Lowes

Iter:   166 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:  101 Obj: -9.86E+01 Gap:  1.22E-02
Iter:   167 I: -1 Tm:      0.00 NLPi:    2 Dpth:    8 Lvs:  100 Obj: -9.88E+01 Gap:  1.22E-02
Iter:   168 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  102 Obj: -9.78E+01 Gap:  1.22E-02
Iter:   169 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  103 Obj: -9.88E+01 Gap:  1.22E-02
--Integer Solution:  -9.77E+01 Lowest Leaf:  -9.88E+01 Gap:   1.15E-02
Iter:   170 I:  0 Tm:      0.00 NLPi:    2 Dpth:    5 Lvs:  102 Obj: -9.50E+01 Gap:  1.15E-02
Iter:   171 I:  0 Tm:      0.00 NLPi:    4 Dpth:    5 Lvs:  101 Obj: -9.66E+01 Gap:  1.15E-02
Iter:   172 I:  0 Tm:      0.00 NLPi:    4 Dpth:    5 Lvs:  103 Obj: -9.88E+01 Gap:  1.15E-02
Iter:   173 I: -1 Tm:      0.00 NLPi:    1 Dpth:   10 Lvs:  102 Obj: -9.88E+01 Gap:  1.15E-02
--Integer Solution:  -9.77E+01 Lowest Leaf:  -9.88E+01 Gap:   1.14E-02
Iter:   174 I:  0 Tm:      0.00 NLPi:    2 Dpth:   10 Lvs:  101 Obj: -7.91E+01 Gap:  1.14E-02
Iter:   175 

In [72]:
m.Equation(x1+x5+x11+x13+x15+x30<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           64
   Intermediates:            0
   Connections  :            0
   Equations    :           30
   Residuals    :           30
 
 Number of state variables:             64
 Number of total equations: -           29
 Number of slack variables: -           26
 ---------------------------------------
 Degrees of freedom       :              9
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -1.00E+02 Gap:       NaN
--Integer Solution:  -9.59E+01 Lowes

Iter:   153 I: -1 Tm:      0.00 NLPi:    3 Dpth:   11 Lvs:   96 Obj: -9.89E+01 Gap:  1.38E-02
--Integer Solution:  -9.76E+01 Lowest Leaf:  -9.89E+01 Gap:   1.36E-02
Iter:   154 I:  0 Tm:      0.00 NLPi:    3 Dpth:   11 Lvs:   95 Obj: -9.04E+01 Gap:  1.36E-02
Iter:   155 I: -1 Tm:      0.00 NLPi:    2 Dpth:    5 Lvs:   94 Obj: -9.89E+01 Gap:  1.36E-02
Iter:   156 I:  0 Tm:      0.00 NLPi:    4 Dpth:    5 Lvs:   96 Obj: -9.86E+01 Gap:  1.36E-02
Iter:   157 I:  0 Tm:      0.00 NLPi:    4 Dpth:    5 Lvs:   95 Obj: -9.72E+01 Gap:  1.36E-02
Iter:   158 I: -1 Tm:      0.00 NLPi:    1 Dpth:    5 Lvs:   94 Obj: -9.89E+01 Gap:  1.36E-02
Iter:   159 I:  0 Tm:      0.00 NLPi:    3 Dpth:    5 Lvs:   96 Obj: -9.85E+01 Gap:  1.36E-02
Iter:   160 I:  0 Tm:      0.00 NLPi:    4 Dpth:    5 Lvs:   98 Obj: -9.82E+01 Gap:  1.36E-02
Iter:   161 I: -1 Tm:      0.00 NLPi:    1 Dpth:    7 Lvs:   97 Obj: -9.89E+01 Gap:  1.36E-02
Iter:   162 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:   96 Obj: -9.70E+01 Gap:

x1: 0.0 Tom Brady
x2: 1.0 Tom Brady /King
x3: 0.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 1.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 0.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 0.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 1.0 Antonio Brown
x12: 0.0 Antonio Brown /King
x13: 1.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 0.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 0.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 0.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 0.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 0.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x31: 1.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 0.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 1.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


In [73]:
m.Equation(x2+x5+x11+x13+x31+x35<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           65
   Intermediates:            0
   Connections  :            0
   Equations    :           31
   Residuals    :           31
 
 Number of state variables:             65
 Number of total equations: -           30
 Number of slack variables: -           27
 ---------------------------------------
 Degrees of freedom       :              8
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -1.00E+02 Gap:       NaN
--Integer Solution:  -9.59E+01 Lowes

Iter:   150 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:   92 Obj: -9.88E+01 Gap:  1.48E-02
--Integer Solution:  -9.74E+01 Lowest Leaf:  -9.89E+01 Gap:   1.45E-02
Iter:   151 I:  0 Tm:      0.00 NLPi:    2 Dpth:   10 Lvs:   91 Obj: -8.36E+01 Gap:  1.45E-02
Iter:   152 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:   92 Obj: -9.88E+01 Gap:  1.45E-02
Iter:   153 I: -1 Tm:      0.00 NLPi:    1 Dpth:    9 Lvs:   91 Obj: -9.89E+01 Gap:  1.45E-02
Iter:   154 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:   90 Obj: -9.69E+01 Gap:  1.45E-02
Iter:   155 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:   91 Obj: -9.88E+01 Gap:  1.45E-02
--Integer Solution:  -9.74E+01 Lowest Leaf:  -9.88E+01 Gap:   1.42E-02
Iter:   156 I:  0 Tm:      0.00 NLPi:    2 Dpth:    7 Lvs:   90 Obj: -9.55E+01 Gap:  1.42E-02
Iter:   157 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:   92 Obj: -9.85E+01 Gap:  1.42E-02
Iter:   158 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:   94 Obj: -9.83E+01 Gap:  1.42E-02
Iter:   159 

x1: 1.0 Tom Brady
x2: 0.0 Tom Brady /King
x3: 0.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 0.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 1.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 0.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 0.0 Antonio Brown
x12: 1.0 Antonio Brown /King
x13: 1.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 1.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 0.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 0.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 0.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 0.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x31: 0.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 0.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 1.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


In [74]:
m.Equation(x1+x7+x12+x13+x15+x35<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           66
   Intermediates:            0
   Connections  :            0
   Equations    :           32
   Residuals    :           32
 
 Number of state variables:             66
 Number of total equations: -           31
 Number of slack variables: -           28
 ---------------------------------------
 Degrees of freedom       :              7
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -1.00E+02 Gap:       NaN
--Integer Solution:  -9.59E+01 Lowes

Iter:   148 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:  102 Obj: -9.35E+01 Gap:  1.35E-02
--Integer Solution:  -9.76E+01 Lowest Leaf:  -9.89E+01 Gap:   1.32E-02
Iter:   149 I:  0 Tm:      0.00 NLPi:    2 Dpth:    7 Lvs:  101 Obj: -9.39E+01 Gap:  1.32E-02
Iter:   150 I:  0 Tm:      0.00 NLPi:    4 Dpth:    7 Lvs:  102 Obj: -9.87E+01 Gap:  1.32E-02
Iter:   151 I: -1 Tm:      0.00 NLPi:    1 Dpth:    7 Lvs:  101 Obj: -9.89E+01 Gap:  1.32E-02
--Integer Solution:  -9.76E+01 Lowest Leaf:  -9.89E+01 Gap:   1.31E-02
Iter:   152 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:  100 Obj: -9.48E+01 Gap:  1.31E-02
Iter:   153 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:  101 Obj: -9.80E+01 Gap:  1.31E-02
Iter:   154 I: -1 Tm:      0.00 NLPi:    1 Dpth:    6 Lvs:  100 Obj: -9.89E+01 Gap:  1.31E-02
Iter:   155 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   99 Obj: -9.70E+01 Gap:  1.31E-02
Iter:   156 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  101 Obj: -9.86E+01 Gap:  1.31E-02
--Integer So

x1: 1.0 Tom Brady
x2: 0.0 Tom Brady /King
x3: 0.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 1.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 1.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 0.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 0.0 Antonio Brown
x12: 1.0 Antonio Brown /King
x13: 0.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 0.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 0.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 0.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 0.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 1.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x31: 1.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 0.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 0.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


In [75]:
m.Equation(x1+x5+x7+x12+x29+x31<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           67
   Intermediates:            0
   Connections  :            0
   Equations    :           33
   Residuals    :           33
 
 Number of state variables:             67
 Number of total equations: -           32
 Number of slack variables: -           29
 ---------------------------------------
 Degrees of freedom       :              6
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -1.00E+02 Gap:       NaN
--Integer Solution:  -9.59E+01 Lowes

Iter:   146 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:  111 Obj: -9.81E+01 Gap:  1.49E-02
Iter:   147 I: -1 Tm:      0.00 NLPi:    1 Dpth:    4 Lvs:  110 Obj: -9.89E+01 Gap:  1.49E-02
Iter:   148 I:  0 Tm:      0.00 NLPi:    2 Dpth:    4 Lvs:  112 Obj: -9.89E+01 Gap:  1.49E-02
Iter:   149 I:  0 Tm:      0.00 NLPi:    4 Dpth:    4 Lvs:  114 Obj: -9.87E+01 Gap:  1.49E-02
Iter:   150 I:  0 Tm:      0.00 NLPi:    3 Dpth:   11 Lvs:  116 Obj: -9.88E+01 Gap:  1.49E-02
Iter:   151 I:  0 Tm:      0.00 NLPi:    4 Dpth:   11 Lvs:  115 Obj: -9.44E+01 Gap:  1.49E-02
Iter:   152 I: -1 Tm:      0.00 NLPi:    1 Dpth:    9 Lvs:  114 Obj: -9.89E+01 Gap:  1.49E-02
--Integer Solution:  -9.74E+01 Lowest Leaf:  -9.89E+01 Gap:   1.46E-02
Iter:   153 I:  0 Tm:      0.00 NLPi:    4 Dpth:    9 Lvs:  113 Obj: -8.63E+01 Gap:  1.46E-02
Iter:   154 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:  114 Obj: -9.88E+01 Gap:  1.46E-02
Iter:   155 I: -1 Tm:      0.00 NLPi:    1 Dpth:    5 Lvs:  113 Obj: -9.89E+01 Gap:

Iter:   238 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:  139 Obj: -9.74E+01 Gap:  1.19E-02
--Integer Solution:  -9.74E+01 Lowest Leaf:  -9.86E+01 Gap:   1.14E-02
Iter:   239 I:  0 Tm:      0.00 NLPi:    2 Dpth:    6 Lvs:  138 Obj: -9.53E+01 Gap:  1.14E-02
Iter:   240 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:  140 Obj: -9.82E+01 Gap:  1.14E-02
Iter:   241 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:  142 Obj: -9.84E+01 Gap:  1.14E-02
Iter:   242 I: -1 Tm:      0.00 NLPi:    1 Dpth:   11 Lvs:  141 Obj: -9.86E+01 Gap:  1.14E-02
Iter:   243 I:  0 Tm:      0.00 NLPi:    2 Dpth:   11 Lvs:  140 Obj: -9.66E+01 Gap:  1.14E-02
Iter:   244 I:  0 Tm:      0.00 NLPi:    3 Dpth:   11 Lvs:  141 Obj: -9.85E+01 Gap:  1.14E-02
--Integer Solution:  -9.74E+01 Lowest Leaf:  -9.86E+01 Gap:   1.13E-02
Iter:   245 I:  0 Tm:      0.00 NLPi:    2 Dpth:    6 Lvs:  140 Obj: -9.67E+01 Gap:  1.13E-02
Iter:   246 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  139 Obj: -9.72E+01 Gap:  1.13E-02
Iter:   247 

In [76]:
m.Equation(x1+x3+x11+x14+x29+x31<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           68
   Intermediates:            0
   Connections  :            0
   Equations    :           34
   Residuals    :           34
 
 Number of state variables:             68
 Number of total equations: -           33
 Number of slack variables: -           30
 ---------------------------------------
 Degrees of freedom       :              5
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -1.00E+02 Gap:       NaN
--Integer Solution:  -9.59E+01 Lowes

Iter:   140 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:   98 Obj: -9.88E+01 Gap:  1.56E-02
Iter:   141 I: -1 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:   97 Obj: -9.90E+01 Gap:  1.56E-02
Iter:   142 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:   99 Obj: -9.78E+01 Gap:  1.56E-02
Iter:   143 I: -1 Tm:      0.00 NLPi:    1 Dpth:    7 Lvs:   98 Obj: -9.89E+01 Gap:  1.56E-02
--Integer Solution:  -9.74E+01 Lowest Leaf:  -9.89E+01 Gap:   1.52E-02
Iter:   144 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:   97 Obj: -8.71E+01 Gap:  1.52E-02
Iter:   145 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:   98 Obj: -9.87E+01 Gap:  1.52E-02
Iter:   146 I: -1 Tm:      0.00 NLPi:    2 Dpth:    5 Lvs:   97 Obj: -9.89E+01 Gap:  1.52E-02
Iter:   147 I:  0 Tm:      0.00 NLPi:    3 Dpth:    5 Lvs:   96 Obj: -9.71E+01 Gap:  1.52E-02
Iter:   148 I:  0 Tm:      0.00 NLPi:    3 Dpth:    5 Lvs:   97 Obj: -9.87E+01 Gap:  1.52E-02
Iter:   149 I: -1 Tm:      0.00 NLPi:    1 Dpth:    6 Lvs:   96 Obj: -9.89E+01 Gap:

--Integer Solution:  -9.74E+01 Lowest Leaf:  -9.86E+01 Gap:   1.14E-02
Iter:   232 I:  0 Tm:      0.00 NLPi:    2 Dpth:    8 Lvs:  134 Obj: -9.69E+01 Gap:  1.14E-02
Iter:   233 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  136 Obj: -9.84E+01 Gap:  1.14E-02
Iter:   234 I:  0 Tm:      0.00 NLPi:    2 Dpth:    8 Lvs:  137 Obj: -9.84E+01 Gap:  1.14E-02
Iter:   235 I: -1 Tm:      0.00 NLPi:    1 Dpth:   10 Lvs:  136 Obj: -9.86E+01 Gap:  1.14E-02
Iter:   236 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:  138 Obj: -9.85E+01 Gap:  1.14E-02
Iter:   237 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:  140 Obj: -9.80E+01 Gap:  1.14E-02
Iter:   238 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  142 Obj: -9.81E+01 Gap:  1.14E-02
--Integer Solution:  -9.74E+01 Lowest Leaf:  -9.85E+01 Gap:   1.11E-02
Iter:   239 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  141 Obj: -9.61E+01 Gap:  1.11E-02
Iter:   240 I:  0 Tm:      0.00 NLPi:    4 Dpth:    7 Lvs:  143 Obj: -9.83E+01 Gap:  1.11E-02
Iter:   241 

In [77]:
m.Equation(x1+x5+x12+x13+x29+x31<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           69
   Intermediates:            0
   Connections  :            0
   Equations    :           35
   Residuals    :           35
 
 Number of state variables:             69
 Number of total equations: -           34
 Number of slack variables: -           31
 ---------------------------------------
 Degrees of freedom       :              4
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -1.00E+02 Gap:       NaN
--Integer Solution:  -9.59E+01 Lowes

Iter:   141 I: -1 Tm:      0.00 NLPi:    1 Dpth:    8 Lvs:   92 Obj: -9.90E+01 Gap:  1.60E-02
--Integer Solution:  -9.74E+01 Lowest Leaf:  -9.90E+01 Gap:   1.59E-02
Iter:   142 I:  0 Tm:      0.00 NLPi:    4 Dpth:    8 Lvs:   91 Obj: -8.89E+01 Gap:  1.59E-02
Iter:   143 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:   92 Obj: -9.90E+01 Gap:  1.59E-02
Iter:   144 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:   94 Obj: -9.89E+01 Gap:  1.59E-02
Iter:   145 I:  0 Tm:      0.00 NLPi:    4 Dpth:    9 Lvs:   96 Obj: -9.80E+01 Gap:  1.59E-02
Iter:   146 I:  0 Tm:      0.00 NLPi:    4 Dpth:    9 Lvs:   97 Obj: -9.89E+01 Gap:  1.59E-02
Iter:   147 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:   96 Obj: -9.51E+01 Gap:  1.59E-02
Iter:   148 I: -1 Tm:      0.00 NLPi:    1 Dpth:   10 Lvs:   95 Obj: -9.89E+01 Gap:  1.59E-02
Iter:   149 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:   97 Obj: -9.88E+01 Gap:  1.59E-02
Iter:   150 I:  0 Tm:      0.00 NLPi:    4 Dpth:   10 Lvs:   96 Obj: -9.44E+01 Gap:

Iter:   243 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:  138 Obj: -9.83E+01 Gap:  1.20E-02
--Integer Solution:  -9.74E+01 Lowest Leaf:  -9.86E+01 Gap:   1.19E-02
Iter:   244 I:  0 Tm:      0.00 NLPi:    2 Dpth:    8 Lvs:  137 Obj: -9.69E+01 Gap:  1.19E-02
Iter:   245 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  139 Obj: -9.80E+01 Gap:  1.19E-02
Iter:   246 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  141 Obj: -9.84E+01 Gap:  1.19E-02
--Integer Solution:  -9.74E+01 Lowest Leaf:  -9.86E+01 Gap:   1.19E-02
Iter:   247 I:  0 Tm:      0.00 NLPi:    2 Dpth:    6 Lvs:  140 Obj: -9.53E+01 Gap:  1.19E-02
Iter:   248 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  142 Obj: -9.83E+01 Gap:  1.19E-02
Iter:   249 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:  144 Obj: -9.75E+01 Gap:  1.19E-02
Iter:   250 I: -1 Tm:      0.00 NLPi:    1 Dpth:    9 Lvs:  143 Obj: -9.86E+01 Gap:  1.19E-02
--Integer Solution:  -9.74E+01 Lowest Leaf:  -9.86E+01 Gap:   1.18E-02
Iter:   251 I:  0 Tm:      0.00 NLP

In [78]:
m.Equation(x2+x3+x11+x15+x29+x35<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           70
   Intermediates:            0
   Connections  :            0
   Equations    :           36
   Residuals    :           36
 
 Number of state variables:             70
 Number of total equations: -           35
 Number of slack variables: -           32
 ---------------------------------------
 Degrees of freedom       :              3
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -1.00E+02 Gap:       NaN
--Integer Solution:  -9.59E+01 Lowes

Iter:   183 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  154 Obj: -8.02E+01 Gap:  1.69E-02
Iter:   184 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  155 Obj: -9.85E+01 Gap:  1.69E-02
Iter:   185 I: -1 Tm:      0.00 NLPi:    2 Dpth:    6 Lvs:  154 Obj: -9.87E+01 Gap:  1.69E-02
Iter:   186 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  156 Obj: -9.75E+01 Gap:  1.69E-02
Iter:   187 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  158 Obj: -9.83E+01 Gap:  1.69E-02
Iter:   188 I: -1 Tm:      0.00 NLPi:    1 Dpth:    6 Lvs:  157 Obj: -9.87E+01 Gap:  1.69E-02
Iter:   189 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:  159 Obj: -9.72E+01 Gap:  1.69E-02
Iter:   190 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:  161 Obj: -9.84E+01 Gap:  1.69E-02
Iter:   191 I: -1 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:  160 Obj: -9.87E+01 Gap:  1.69E-02
Iter:   192 I:  0 Tm:      0.00 NLPi:    5 Dpth:    9 Lvs:  159 Obj: -9.52E+01 Gap:  1.69E-02
Iter:   193 I: -1 Tm:      0.00 NLPi:    1 Dpth:    8 Lvs:  

Iter:   307 I:  0 Tm:      0.00 NLPi:    3 Dpth:   13 Lvs:  188 Obj: -9.82E+01 Gap:  1.27E-02
Iter:   308 I: -1 Tm:      0.00 NLPi:    1 Dpth:    6 Lvs:  187 Obj: -9.83E+01 Gap:  1.27E-02
Iter:   309 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:  186 Obj: -9.64E+01 Gap:  1.27E-02
Iter:   310 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  187 Obj: -9.83E+01 Gap:  1.27E-02
Iter:   311 I: -1 Tm:      0.00 NLPi:    1 Dpth:    9 Lvs:  186 Obj: -9.83E+01 Gap:  1.27E-02
--Integer Solution:  -9.70E+01 Lowest Leaf:  -9.83E+01 Gap:   1.25E-02
Iter:   312 I:  0 Tm:      0.00 NLPi:    2 Dpth:    9 Lvs:  185 Obj: -8.30E+01 Gap:  1.25E-02
Iter:   313 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:  186 Obj: -9.81E+01 Gap:  1.25E-02
--Integer Solution:  -9.70E+01 Lowest Leaf:  -9.83E+01 Gap:   1.25E-02
Iter:   314 I:  0 Tm:      0.00 NLPi:    2 Dpth:    7 Lvs:  185 Obj: -9.55E+01 Gap:  1.25E-02
Iter:   315 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:  184 Obj: -9.64E+01 Gap:  1.25E-02
Iter:   316 

x1: 0.0 Tom Brady
x2: 1.0 Tom Brady /King
x3: 0.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 1.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 1.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 0.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 1.0 Antonio Brown
x12: 0.0 Antonio Brown /King
x13: 0.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 0.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 0.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 0.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 0.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 0.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x31: 1.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 0.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 1.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


In [79]:
m.Equation(x2+x5+x7+x11+x31+x35<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           71
   Intermediates:            0
   Connections  :            0
   Equations    :           37
   Residuals    :           37
 
 Number of state variables:             71
 Number of total equations: -           36
 Number of slack variables: -           33
 ---------------------------------------
 Degrees of freedom       :              2
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -1.00E+02 Gap:       NaN
--Integer Solution:  -9.59E+01 Lowes

Iter:   175 I:  0 Tm:      0.00 NLPi:    4 Dpth:    8 Lvs:  141 Obj: -9.86E+01 Gap:  1.82E-02
Iter:   176 I:  0 Tm:      0.00 NLPi:    4 Dpth:    8 Lvs:  143 Obj: -9.82E+01 Gap:  1.82E-02
Iter:   177 I: -1 Tm:      0.00 NLPi:    1 Dpth:    8 Lvs:  142 Obj: -9.87E+01 Gap:  1.82E-02
Iter:   178 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  141 Obj: -9.65E+01 Gap:  1.82E-02
Iter:   179 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  143 Obj: -9.82E+01 Gap:  1.82E-02
Iter:   180 I: -1 Tm:      0.00 NLPi:    1 Dpth:    7 Lvs:  142 Obj: -9.87E+01 Gap:  1.82E-02
Iter:   181 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:  143 Obj: -9.87E+01 Gap:  1.82E-02
Iter:   182 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:  142 Obj: -9.57E+01 Gap:  1.82E-02
--Integer Solution:  -9.70E+01 Lowest Leaf:  -9.87E+01 Gap:   1.70E-02
Iter:   183 I:  0 Tm:      0.00 NLPi:    2 Dpth:    6 Lvs:  141 Obj: -9.64E+01 Gap:  1.70E-02
Iter:   184 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  143 Obj: -9.72E+01 Gap:

Iter:   271 I:  0 Tm:      0.00 NLPi:    4 Dpth:    9 Lvs:  182 Obj: -9.73E+01 Gap:  1.23E-02
Iter:   272 I: -1 Tm:      0.00 NLPi:    1 Dpth:    7 Lvs:  181 Obj: -9.84E+01 Gap:  1.23E-02
Iter:   273 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:  183 Obj: -9.76E+01 Gap:  1.23E-02
Iter:   274 I:  0 Tm:      0.00 NLPi:    4 Dpth:    7 Lvs:  185 Obj: -9.81E+01 Gap:  1.23E-02
Iter:   275 I: -1 Tm:      0.00 NLPi:    1 Dpth:    6 Lvs:  184 Obj: -9.84E+01 Gap:  1.23E-02
Iter:   276 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  183 Obj: -9.72E+01 Gap:  1.23E-02
Iter:   277 I:  0 Tm:      0.00 NLPi:    5 Dpth:    6 Lvs:  185 Obj: -9.80E+01 Gap:  1.23E-02
Iter:   278 I: -1 Tm:      0.00 NLPi:    1 Dpth:    8 Lvs:  184 Obj: -9.84E+01 Gap:  1.23E-02
Iter:   279 I: -1 Tm:      0.00 NLPi:    1 Dpth:    8 Lvs:  183 Obj: -9.84E+01 Gap:  1.23E-02
Iter:   280 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  182 Obj: -9.67E+01 Gap:  1.23E-02
Iter:   281 I:  0 Tm:      0.00 NLPi:    2 Dpth:    6 Lvs:  

x1: 1.0 Tom Brady
x2: 0.0 Tom Brady /King
x3: 0.0 Jalen Hurts
x4: 1.0 Jalen Hurts /King
x5: 0.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 0.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 0.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 1.0 Antonio Brown
x12: 0.0 Antonio Brown /King
x13: 1.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 0.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 0.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 0.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 0.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 0.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x31: 1.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 0.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 1.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


In [80]:
m.Equation(x1+x4+x11+x13+x31+x35<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           72
   Intermediates:            0
   Connections  :            0
   Equations    :           38
   Residuals    :           38
 
 Number of state variables:             72
 Number of total equations: -           37
 Number of slack variables: -           34
 ---------------------------------------
 Degrees of freedom       :              1
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -1.00E+02 Gap:       NaN
--Integer Solution:  -9.59E+01 Lowes

Iter:   133 I: -1 Tm:      0.00 NLPi:    2 Dpth:    6 Lvs:  100 Obj: -9.89E+01 Gap:  1.93E-02
Iter:   134 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:  102 Obj: -9.88E+01 Gap:  1.93E-02
Iter:   135 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  104 Obj: -9.81E+01 Gap:  1.93E-02
Iter:   136 I:  0 Tm:      0.00 NLPi:    3 Dpth:   15 Lvs:  106 Obj: -9.88E+01 Gap:  1.93E-02
Iter:   137 I:  0 Tm:      0.00 NLPi:    3 Dpth:   15 Lvs:  105 Obj: -9.60E+01 Gap:  1.93E-02
Iter:   138 I: -1 Tm:      0.00 NLPi:    1 Dpth:    6 Lvs:  104 Obj: -9.89E+01 Gap:  1.93E-02
Iter:   139 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  106 Obj: -9.77E+01 Gap:  1.93E-02
Iter:   140 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:  108 Obj: -9.83E+01 Gap:  1.93E-02
--Integer Solution:  -9.70E+01 Lowest Leaf:  -9.89E+01 Gap:   1.87E-02
Iter:   141 I:  0 Tm:      0.00 NLPi:    4 Dpth:    8 Lvs:  107 Obj: -8.70E+01 Gap:  1.87E-02
Iter:   142 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  108 Obj: -9.86E+01 Gap:

Iter:   246 I: -1 Tm:      0.00 NLPi:    1 Dpth:    8 Lvs:  170 Obj: -9.84E+01 Gap:  1.24E-02
--Integer Solution:  -9.72E+01 Lowest Leaf:  -9.84E+01 Gap:   1.23E-02
Iter:   247 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  169 Obj: -9.26E+01 Gap:  1.23E-02
Iter:   248 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  170 Obj: -9.83E+01 Gap:  1.23E-02
Iter:   249 I:  0 Tm:      0.00 NLPi:    2 Dpth:    6 Lvs:  172 Obj: -9.87E+01 Gap:  1.23E-02
Iter:   250 I: -1 Tm:      0.00 NLPi:    2 Dpth:    7 Lvs:  171 Obj: -9.87E+01 Gap:  1.23E-02
--Integer Solution:  -9.72E+01 Lowest Leaf:  -9.87E+01 Gap:   1.49E-02
Iter:   251 I:  0 Tm:      0.00 NLPi:    2 Dpth:    7 Lvs:  170 Obj: -9.17E+01 Gap:  1.49E-02
Iter:   252 I: -1 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:  169 Obj: -9.87E+01 Gap:  1.49E-02
Iter:   253 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:  171 Obj: -9.83E+01 Gap:  1.49E-02
Iter:   254 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  173 Obj: -9.81E+01 Gap:  1.49E-02
Iter:   255 

x1: 1.0 Tom Brady
x2: 0.0 Tom Brady /King
x3: 0.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 0.0 Mike Evans
x6: 1.0 Mike Evans /King
x7: 0.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 0.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 1.0 Antonio Brown
x12: 0.0 Antonio Brown /King
x13: 1.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 0.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 0.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 0.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 0.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 1.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x31: 1.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 0.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 0.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


In [81]:
m.Equation(x1+x6+x11+x13+x29+x31<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           73
   Intermediates:            0
   Connections  :            0
   Equations    :           39
   Residuals    :           39
 
 Number of state variables:             73
 Number of total equations: -           38
 Number of slack variables: -           35
 ---------------------------------------
 Degrees of freedom       :              0
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.01 NLPi:    6 Dpth:    0 Lvs:    3 Obj: -1.00E+02 Gap:       NaN
--Integer Solution:  -9.59E+01 Lowes

Iter:   117 I:  0 Tm:      0.00 NLPi:    4 Dpth:    5 Lvs:   97 Obj: -9.80E+01 Gap:  2.56E-02
Iter:   118 I:  0 Tm:      0.00 NLPi:    4 Dpth:    7 Lvs:   99 Obj: -9.85E+01 Gap:  2.56E-02
Iter:   119 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:  101 Obj: -9.86E+01 Gap:  2.56E-02
--Integer Solution:  -9.65E+01 Lowest Leaf:  -9.90E+01 Gap:   2.52E-02
Iter:   120 I:  0 Tm:      0.00 NLPi:    4 Dpth:    9 Lvs:  100 Obj: -8.45E+01 Gap:  2.52E-02
Iter:   121 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:  101 Obj: -9.90E+01 Gap:  2.52E-02
Iter:   122 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:  103 Obj: -9.90E+01 Gap:  2.52E-02
Iter:   123 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:  105 Obj: -9.78E+01 Gap:  2.52E-02
Iter:   124 I: -1 Tm:      0.00 NLPi:    1 Dpth:   11 Lvs:  104 Obj: -9.90E+01 Gap:  2.52E-02
--Integer Solution:  -9.65E+01 Lowest Leaf:  -9.90E+01 Gap:   2.50E-02
Iter:   125 I:  0 Tm:      0.00 NLPi:    4 Dpth:   11 Lvs:  103 Obj: -8.95E+01 Gap:  2.50E-02
Iter:   126 

Iter:   221 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  158 Obj: -9.80E+01 Gap:  1.71E-02
Iter:   222 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  160 Obj: -9.84E+01 Gap:  1.71E-02
Iter:   223 I: -1 Tm:      0.00 NLPi:    2 Dpth:    8 Lvs:  159 Obj: -9.85E+01 Gap:  1.71E-02
--Integer Solution:  -9.69E+01 Lowest Leaf:  -9.85E+01 Gap:   1.68E-02
Iter:   224 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  158 Obj: -9.23E+01 Gap:  1.68E-02
Iter:   225 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  159 Obj: -9.84E+01 Gap:  1.68E-02
Iter:   226 I: -1 Tm:      0.00 NLPi:    1 Dpth:    9 Lvs:  158 Obj: -9.85E+01 Gap:  1.68E-02
Iter:   227 I:  0 Tm:      0.00 NLPi:    4 Dpth:    9 Lvs:  157 Obj: -9.56E+01 Gap:  1.68E-02
Iter:   228 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:  159 Obj: -9.83E+01 Gap:  1.68E-02
--Integer Solution:  -9.69E+01 Lowest Leaf:  -9.85E+01 Gap:   1.67E-02
Iter:   229 I:  0 Tm:      0.00 NLPi:    2 Dpth:    7 Lvs:  158 Obj: -9.62E+01 Gap:  1.67E-02
Iter:   230 

Iter:   403 I:  0 Tm:      0.00 NLPi:    3 Dpth:   11 Lvs:  189 Obj: -9.63E+01 Gap:  1.14E-02
Iter:   404 I:  0 Tm:      0.00 NLPi:    3 Dpth:   11 Lvs:  190 Obj: -9.79E+01 Gap:  1.14E-02
--Integer Solution:  -9.69E+01 Lowest Leaf:  -9.80E+01 Gap:   1.13E-02
Iter:   405 I:  0 Tm:      0.00 NLPi:    2 Dpth:   18 Lvs:  189 Obj: -8.28E+01 Gap:  1.13E-02
Iter:   406 I:  0 Tm:      0.00 NLPi:    4 Dpth:   18 Lvs:  190 Obj: -9.80E+01 Gap:  1.13E-02
Iter:   407 I: -1 Tm:      0.00 NLPi:    1 Dpth:   13 Lvs:  189 Obj: -9.80E+01 Gap:  1.13E-02
Iter:   408 I:  0 Tm:      0.00 NLPi:    3 Dpth:   13 Lvs:  191 Obj: -9.79E+01 Gap:  1.13E-02
Iter:   409 I:  0 Tm:      0.00 NLPi:    4 Dpth:   13 Lvs:  190 Obj: -9.43E+01 Gap:  1.13E-02
Iter:   410 I: -1 Tm:      0.00 NLPi:    1 Dpth:   10 Lvs:  189 Obj: -9.80E+01 Gap:  1.13E-02
Iter:   411 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:  191 Obj: -9.79E+01 Gap:  1.13E-02
Iter:   412 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:  193 Obj: -9.74E+01 Gap:

In [82]:
m.Equation(x1+x5+x11+x13+x15+x32<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           74
   Intermediates:            0
   Connections  :            0
   Equations    :           40
   Residuals    :           40
 
 Number of state variables:             74
 Number of total equations: -           39
 Number of slack variables: -           36
 ---------------------------------------
 Degrees of freedom       :             -1
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -1.00E+02 Gap:       NaN
--Integer Solut

Iter:   137 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  103 Obj: -9.83E+01 Gap:  2.73E-02
Iter:   138 I: -1 Tm:      0.00 NLPi:    1 Dpth:    5 Lvs:  102 Obj: -9.89E+01 Gap:  2.73E-02
Iter:   139 I:  0 Tm:      0.00 NLPi:    3 Dpth:    5 Lvs:  104 Obj: -9.84E+01 Gap:  2.73E-02
Iter:   140 I:  0 Tm:      0.00 NLPi:    3 Dpth:    5 Lvs:  106 Obj: -9.83E+01 Gap:  2.73E-02
Iter:   141 I: -1 Tm:      0.00 NLPi:    1 Dpth:    8 Lvs:  105 Obj: -9.89E+01 Gap:  2.73E-02
Iter:   142 I:  0 Tm:      0.00 NLPi:    2 Dpth:    8 Lvs:  107 Obj: -9.71E+01 Gap:  2.73E-02
Iter:   143 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  108 Obj: -9.87E+01 Gap:  2.73E-02
--Integer Solution:  -9.70E+01 Lowest Leaf:  -9.88E+01 Gap:   1.82E-02
Iter:   144 I:  0 Tm:      0.00 NLPi:    2 Dpth:    6 Lvs:  107 Obj: -9.70E+01 Gap:  1.82E-02
Iter:   145 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:  109 Obj: -9.72E+01 Gap:  1.82E-02
Iter:   146 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  110 Obj: -9.85E+01 Gap:

Iter:   240 I:  0 Tm:      0.00 NLPi:    3 Dpth:    5 Lvs:  164 Obj: -9.81E+01 Gap:  1.48E-02
Iter:   241 I:  0 Tm:      0.00 NLPi:    3 Dpth:   11 Lvs:  165 Obj: -9.84E+01 Gap:  1.48E-02
Iter:   242 I:  0 Tm:      0.00 NLPi:    4 Dpth:   11 Lvs:  164 Obj: -9.50E+01 Gap:  1.48E-02
Iter:   243 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:  166 Obj: -9.81E+01 Gap:  1.48E-02
Iter:   244 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:  168 Obj: -9.74E+01 Gap:  1.48E-02
Iter:   245 I: -1 Tm:      0.00 NLPi:    1 Dpth:   12 Lvs:  167 Obj: -9.85E+01 Gap:  1.48E-02
Iter:   246 I:  0 Tm:      0.00 NLPi:    3 Dpth:   12 Lvs:  166 Obj: -9.37E+01 Gap:  1.48E-02
Iter:   247 I: -1 Tm:      0.00 NLPi:    3 Dpth:   12 Lvs:  165 Obj: -9.85E+01 Gap:  1.48E-02
Iter:   248 I:  0 Tm:      0.00 NLPi:    2 Dpth:   12 Lvs:  164 Obj: -9.61E+01 Gap:  1.48E-02
Iter:   249 I:  0 Tm:      0.00 NLPi:    3 Dpth:   12 Lvs:  165 Obj: -9.82E+01 Gap:  1.48E-02
Iter:   250 I: -1 Tm:      0.00 NLPi:    1 Dpth:    6 Lvs:  

--Integer Solution:  -9.70E+01 Lowest Leaf:  -9.82E+01 Gap:   1.13E-02
Iter:   354 I:  0 Tm:      0.00 NLPi:    2 Dpth:    6 Lvs:  199 Obj: -9.64E+01 Gap:  1.13E-02
Iter:   355 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:  201 Obj: -9.74E+01 Gap:  1.13E-02
Iter:   356 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  202 Obj: -9.79E+01 Gap:  1.13E-02
Iter:   357 I: -1 Tm:      0.00 NLPi:    1 Dpth:   11 Lvs:  201 Obj: -9.82E+01 Gap:  1.13E-02
--Integer Solution:  -9.70E+01 Lowest Leaf:  -9.82E+01 Gap:   1.13E-02
Iter:   358 I:  0 Tm:      0.00 NLPi:    2 Dpth:   11 Lvs:  200 Obj: -8.49E+01 Gap:  1.13E-02
Iter:   359 I:  0 Tm:      0.00 NLPi:    3 Dpth:   11 Lvs:  201 Obj: -9.81E+01 Gap:  1.13E-02
--Integer Solution:  -9.70E+01 Lowest Leaf:  -9.81E+01 Gap:   1.12E-02
Iter:   360 I:  0 Tm:      0.00 NLPi:    3 Dpth:   12 Lvs:  200 Obj: -8.29E+01 Gap:  1.12E-02
Iter:   361 I:  0 Tm:      0.00 NLPi:    3 Dpth:   12 Lvs:  201 Obj: -9.81E+01 Gap:  1.12E-02
Iter:   362 I: -1 Tm:      0.00 NLP

In [83]:
m.Equation(x1+x3+x12+x13+x29+x35<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           75
   Intermediates:            0
   Connections  :            0
   Equations    :           41
   Residuals    :           41
 
 Number of state variables:             75
 Number of total equations: -           40
 Number of slack variables: -           37
 ---------------------------------------
 Degrees of freedom       :             -2
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -1.00E+02 Gap:       NaN
--Integer Solut

Iter:   117 I:  0 Tm:      0.00 NLPi:    3 Dpth:    5 Lvs:   95 Obj: -9.91E+01 Gap:  3.32E-02
--Integer Solution:  -9.59E+01 Lowest Leaf:  -9.91E+01 Gap:   3.30E-02
Iter:   118 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:   94 Obj: -9.19E+01 Gap:  3.30E-02
Iter:   119 I:  0 Tm:      0.00 NLPi:    4 Dpth:    9 Lvs:   95 Obj: -9.91E+01 Gap:  3.30E-02
Iter:   120 I: -1 Tm:      0.00 NLPi:    1 Dpth:    7 Lvs:   94 Obj: -9.91E+01 Gap:  3.30E-02
Iter:   121 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:   95 Obj: -9.88E+01 Gap:  3.30E-02
Iter:   122 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:   96 Obj: -9.89E+01 Gap:  3.30E-02
Iter:   123 I: -1 Tm:      0.00 NLPi:    1 Dpth:    6 Lvs:   95 Obj: -9.91E+01 Gap:  3.30E-02
Iter:   124 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   97 Obj: -9.90E+01 Gap:  3.30E-02
Iter:   125 I:  0 Tm:      0.00 NLPi:    7 Dpth:    6 Lvs:   99 Obj: -9.88E+01 Gap:  3.30E-02
Iter:   126 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:  101 Obj: -9.90E+01 Gap:

Iter:   204 I:  0 Tm:      0.00 NLPi:    4 Dpth:    5 Lvs:  143 Obj: -9.88E+01 Gap:  1.92E-02
Iter:   205 I: -1 Tm:      0.00 NLPi:    1 Dpth:   11 Lvs:  142 Obj: -9.88E+01 Gap:  1.92E-02
--Integer Solution:  -9.69E+01 Lowest Leaf:  -9.88E+01 Gap:   1.91E-02
Iter:   206 I:  0 Tm:      0.00 NLPi:    4 Dpth:   11 Lvs:  141 Obj: -8.90E+01 Gap:  1.91E-02
Iter:   207 I:  0 Tm:      0.00 NLPi:    3 Dpth:   11 Lvs:  142 Obj: -9.86E+01 Gap:  1.91E-02
Iter:   208 I:  0 Tm:      0.00 NLPi:    4 Dpth:    9 Lvs:  144 Obj: -9.86E+01 Gap:  1.91E-02
Iter:   209 I:  0 Tm:      0.00 NLPi:    4 Dpth:    9 Lvs:  146 Obj: -9.78E+01 Gap:  1.91E-02
Iter:   210 I: -1 Tm:      0.00 NLPi:    1 Dpth:    6 Lvs:  145 Obj: -9.88E+01 Gap:  1.91E-02
Iter:   211 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  147 Obj: -9.87E+01 Gap:  1.91E-02
Iter:   212 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:  148 Obj: -9.86E+01 Gap:  1.91E-02
Iter:   213 I: -1 Tm:      0.00 NLPi:    1 Dpth:    7 Lvs:  147 Obj: -9.88E+01 Gap:

Iter:   371 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:  204 Obj: -9.80E+01 Gap:  1.34E-02
Iter:   372 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:  206 Obj: -9.76E+01 Gap:  1.34E-02
Iter:   373 I: -1 Tm:      0.00 NLPi:    1 Dpth:    8 Lvs:  205 Obj: -9.82E+01 Gap:  1.34E-02
Iter:   374 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  207 Obj: -9.78E+01 Gap:  1.34E-02
Iter:   375 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  209 Obj: -9.80E+01 Gap:  1.34E-02
Iter:   376 I: -1 Tm:      0.00 NLPi:    1 Dpth:    9 Lvs:  208 Obj: -9.82E+01 Gap:  1.34E-02
--Integer Solution:  -9.69E+01 Lowest Leaf:  -9.82E+01 Gap:   1.32E-02
Iter:   377 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:  207 Obj: -9.23E+01 Gap:  1.32E-02
Iter:   378 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:  208 Obj: -9.81E+01 Gap:  1.32E-02
Iter:   379 I: -1 Tm:      0.00 NLPi:    1 Dpth:    8 Lvs:  207 Obj: -9.82E+01 Gap:  1.32E-02
Iter:   380 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  209 Obj: -9.74E+01 Gap:

Iter:   456 I:  0 Tm:      0.00 NLPi:    4 Dpth:    9 Lvs:  203 Obj: -9.71E+01 Gap:  1.10E-02
Iter:   457 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:  205 Obj: -9.75E+01 Gap:  1.10E-02
Iter:   458 I: -1 Tm:      0.00 NLPi:    2 Dpth:   10 Lvs:  204 Obj: -9.80E+01 Gap:  1.10E-02
--Integer Solution:  -9.69E+01 Lowest Leaf:  -9.80E+01 Gap:   1.08E-02
Iter:   459 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:  203 Obj: -9.33E+01 Gap:  1.08E-02
Iter:   460 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  202 Obj: -9.66E+01 Gap:  1.08E-02
Iter:   461 I:  0 Tm:      0.00 NLPi:    4 Dpth:    8 Lvs:  204 Obj: -9.76E+01 Gap:  1.08E-02
Iter:   462 I: -1 Tm:      0.00 NLPi:    1 Dpth:    8 Lvs:  203 Obj: -9.80E+01 Gap:  1.08E-02
Iter:   463 I:  0 Tm:      0.00 NLPi:    2 Dpth:    8 Lvs:  205 Obj: -9.78E+01 Gap:  1.08E-02
Iter:   464 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  207 Obj: -9.77E+01 Gap:  1.08E-02
Iter:   465 I: -1 Tm:      0.00 NLPi:    1 Dpth:    8 Lvs:  206 Obj: -9.79E+01 Gap:

2222222

In [84]:
m.Equation(x2+x3+x11+x15+x31+x35<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           76
   Intermediates:            0
   Connections  :            0
   Equations    :           42
   Residuals    :           42
 
 Number of state variables:             76
 Number of total equations: -           41
 Number of slack variables: -           38
 ---------------------------------------
 Degrees of freedom       :             -3
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -1.00E+02 Gap:       NaN
--Integer Solut

Iter:   161 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  132 Obj: -9.77E+01 Gap:  2.27E-02
Iter:   162 I: -1 Tm:      0.00 NLPi:    1 Dpth:   10 Lvs:  131 Obj: -9.88E+01 Gap:  2.27E-02
Iter:   163 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:  133 Obj: -9.85E+01 Gap:  2.27E-02
Iter:   164 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:  135 Obj: -9.80E+01 Gap:  2.27E-02
Iter:   165 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  134 Obj: -9.58E+01 Gap:  2.27E-02
Iter:   166 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  136 Obj: -9.81E+01 Gap:  2.27E-02
Iter:   167 I: -1 Tm:      0.00 NLPi:    1 Dpth:    6 Lvs:  135 Obj: -9.87E+01 Gap:  2.27E-02
Iter:   168 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:  137 Obj: -9.86E+01 Gap:  2.27E-02
Iter:   169 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  139 Obj: -9.77E+01 Gap:  2.27E-02
--Integer Solution:  -9.65E+01 Lowest Leaf:  -9.87E+01 Gap:   2.19E-02
Iter:   170 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  138 Obj: -8.75E+01 Gap:

Iter:   270 I:  0 Tm:      0.01 NLPi:   12 Dpth:    6 Lvs:  185 Obj: -9.83E+01 Gap:  1.78E-02
Iter:   271 I: -1 Tm:      0.00 NLPi:    1 Dpth:   16 Lvs:  184 Obj: -9.83E+01 Gap:  1.78E-02
--Integer Solution:  -9.65E+01 Lowest Leaf:  -9.83E+01 Gap:   1.77E-02
Iter:   272 I:  0 Tm:      0.00 NLPi:    3 Dpth:   16 Lvs:  183 Obj: -8.70E+01 Gap:  1.77E-02
Iter:   273 I:  0 Tm:      0.00 NLPi:    2 Dpth:   16 Lvs:  184 Obj: -9.76E+01 Gap:  1.77E-02
Iter:   274 I:  0 Tm:      0.00 NLPi:    5 Dpth:    7 Lvs:  183 Obj: -9.55E+01 Gap:  1.77E-02
Iter:   275 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:  185 Obj: -9.83E+01 Gap:  1.77E-02
Iter:   276 I: -1 Tm:      0.00 NLPi:    1 Dpth:    8 Lvs:  184 Obj: -9.83E+01 Gap:  1.77E-02
Iter:   277 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:  186 Obj: -9.75E+01 Gap:  1.77E-02
Iter:   278 I:  0 Tm:      0.00 NLPi:    5 Dpth:    8 Lvs:  188 Obj: -9.82E+01 Gap:  1.77E-02
--Integer Solution:  -9.65E+01 Lowest Leaf:  -9.83E+01 Gap:   1.76E-02
Iter:   279 

Iter:   436 I:  0 Tm:      0.00 NLPi:    4 Dpth:    8 Lvs:  224 Obj: -8.97E+01 Gap:  1.00E-02
Iter:   437 I:  0 Tm:      0.00 NLPi:    4 Dpth:    8 Lvs:  226 Obj: -9.77E+01 Gap:  1.00E-02
Iter:   438 I:  0 Tm:      0.00 NLPi:    3 Dpth:   14 Lvs:  228 Obj: -9.76E+01 Gap:  1.00E-02
Iter:   439 I:  0 Tm:      0.00 NLPi:    3 Dpth:   14 Lvs:  227 Obj: -9.32E+01 Gap:  1.00E-02
Iter:   440 I:  0 Tm:      0.00 NLPi:    4 Dpth:   13 Lvs:  226 Obj: -9.47E+01 Gap:  1.00E-02
Iter:   441 I:  0 Tm:      0.00 NLPi:    4 Dpth:   13 Lvs:  227 Obj: -9.74E+01 Gap:  1.00E-02
--Integer Solution:  -9.69E+01 Lowest Leaf:  -9.79E+01 Gap:   9.87E-03
Iter:   442 I:  0 Tm:      0.00 NLPi:    2 Dpth:    7 Lvs:  227 Obj: -9.59E+01 Gap:  9.87E-03
 Successful solution
 
 ---------------------------------------------------
 Solver         :  APOPT (v1.0)
 Solution time  :    1.27019999999993      sec
 Objective      :   -96.9000000000000     
 Successful solution
 ---------------------------------------------------

In [85]:
m.Equation(x2+x7+x11+x13+x19+x35<= 5)
m.Equation(x31+x32==0)
m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           77
   Intermediates:            0
   Connections  :            0
   Equations    :           44
   Residuals    :           44
 
 Number of state variables:             77
 Number of total equations: -           43
 Number of slack variables: -           39
 ---------------------------------------
 Degrees of freedom       :             -5
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.01 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -9.94E+01 Gap:       NaN
Iter:     2 I: 

Iter:    99 I:  0 Tm:      0.00 NLPi:    4 Dpth:    7 Lvs:   89 Obj: -9.54E+01 Gap:  2.86E-02
Iter:   100 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:   90 Obj: -9.81E+01 Gap:  2.86E-02
Iter:   101 I:  0 Tm:      0.00 NLPi:    3 Dpth:    5 Lvs:   92 Obj: -9.64E+01 Gap:  2.86E-02
Iter:   102 I:  0 Tm:      0.00 NLPi:    4 Dpth:    5 Lvs:   93 Obj: -9.79E+01 Gap:  2.86E-02
Iter:   103 I: -1 Tm:      0.00 NLPi:    1 Dpth:    5 Lvs:   92 Obj: -9.82E+01 Gap:  2.86E-02
Iter:   104 I:  0 Tm:      0.00 NLPi:    5 Dpth:    5 Lvs:   94 Obj: -9.64E+01 Gap:  2.86E-02
Iter:   105 I:  0 Tm:      0.00 NLPi:    3 Dpth:    5 Lvs:   96 Obj: -9.82E+01 Gap:  2.86E-02
Iter:   106 I: -1 Tm:      0.00 NLPi:    1 Dpth:    5 Lvs:   95 Obj: -9.82E+01 Gap:  2.86E-02
Iter:   107 I:  0 Tm:      0.00 NLPi:    4 Dpth:    5 Lvs:   97 Obj: -9.75E+01 Gap:  2.86E-02
Iter:   108 I:  0 Tm:      0.00 NLPi:    4 Dpth:    5 Lvs:   99 Obj: -9.81E+01 Gap:  2.86E-02
Iter:   109 I: -1 Tm:      0.00 NLPi:    3 Dpth:    5 Lvs:  

Iter:   182 I: -1 Tm:      0.00 NLPi:    3 Dpth:   18 Lvs:  128 Obj: -9.80E+01 Gap:  2.21E-02
Iter:   183 I:  0 Tm:      0.00 NLPi:    3 Dpth:   18 Lvs:  127 Obj: -9.31E+01 Gap:  2.21E-02
--Integer Solution:  -9.58E+01 Lowest Leaf:  -9.79E+01 Gap:   2.13E-02
Iter:   184 I:  0 Tm:      0.00 NLPi:    2 Dpth:    6 Lvs:  126 Obj: -9.58E+01 Gap:  2.13E-02
Iter:   185 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  125 Obj: -9.49E+01 Gap:  2.13E-02
Iter:   186 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  127 Obj: -9.79E+01 Gap:  2.13E-02
--Integer Solution:  -9.58E+01 Lowest Leaf:  -9.79E+01 Gap:   2.09E-02
Iter:   187 I:  0 Tm:      0.00 NLPi:    2 Dpth:    7 Lvs:  126 Obj: -9.58E+01 Gap:  2.09E-02
Iter:   188 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:  127 Obj: -9.78E+01 Gap:  2.09E-02
Iter:   189 I:  0 Tm:      0.00 NLPi:    5 Dpth:    7 Lvs:  129 Obj: -9.75E+01 Gap:  2.09E-02
Iter:   190 I: -1 Tm:      0.00 NLPi:    1 Dpth:    6 Lvs:  128 Obj: -9.79E+01 Gap:  2.09E-02
Iter:   191 

In [86]:
m.Equation(x1+x5+x11+x13+x16+x35<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           78
   Intermediates:            0
   Connections  :            0
   Equations    :           45
   Residuals    :           45
 
 Number of state variables:             78
 Number of total equations: -           44
 Number of slack variables: -           40
 ---------------------------------------
 Degrees of freedom       :             -6
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.01 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -9.95E+01 Gap:       NaN
Iter:     2 I: 

Iter:   105 I:  0 Tm:      0.00 NLPi:    4 Dpth:    4 Lvs:   92 Obj: -9.69E+01 Gap:  2.87E-02
Iter:   106 I:  0 Tm:      0.00 NLPi:    4 Dpth:    4 Lvs:   94 Obj: -9.81E+01 Gap:  2.87E-02
--Integer Solution:  -9.55E+01 Lowest Leaf:  -9.83E+01 Gap:   2.86E-02
Iter:   107 I:  0 Tm:      0.00 NLPi:    2 Dpth:    7 Lvs:   93 Obj: -9.55E+01 Gap:  2.86E-02
Iter:   108 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:   95 Obj: -9.64E+01 Gap:  2.86E-02
Iter:   109 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:   97 Obj: -9.73E+01 Gap:  2.86E-02
Iter:   110 I:  0 Tm:      0.00 NLPi:    3 Dpth:    5 Lvs:   99 Obj: -9.80E+01 Gap:  2.86E-02
Iter:   111 I:  0 Tm:      0.00 NLPi:    3 Dpth:    5 Lvs:  101 Obj: -9.77E+01 Gap:  2.86E-02
Iter:   112 I: -1 Tm:      0.00 NLPi:    2 Dpth:    6 Lvs:  100 Obj: -9.82E+01 Gap:  2.86E-02
Iter:   113 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:  102 Obj: -9.67E+01 Gap:  2.86E-02
Iter:   114 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  103 Obj: -9.81E+01 Gap:

 
 ---------------------------------------------------
 Solver         :  APOPT (v1.0)
 Solution time  :   0.528299999999945      sec
 Objective      :   -96.9000000000000     
 Successful solution
 ---------------------------------------------------
 
x1: 0.0 Tom Brady
x2: 1.0 Tom Brady /King
x3: 0.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 0.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 0.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 0.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 1.0 Antonio Brown
x12: 0.0 Antonio Brown /King
x13: 1.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 1.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 0.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 1.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 0.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 1.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x3

In [87]:
m.Equation(x2+x11+x13+x15+x19+x29<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           79
   Intermediates:            0
   Connections  :            0
   Equations    :           46
   Residuals    :           46
 
 Number of state variables:             79
 Number of total equations: -           45
 Number of slack variables: -           41
 ---------------------------------------
 Degrees of freedom       :             -7
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -9.94E+01 Gap:       NaN
--Integer Solut

Iter:    97 I:  0 Tm:      0.00 NLPi:    3 Dpth:   16 Lvs:   68 Obj: -7.92E+01 Gap:  3.48E-02
Iter:    98 I:  0 Tm:      0.00 NLPi:    3 Dpth:   16 Lvs:   69 Obj: -9.85E+01 Gap:  3.48E-02
Iter:    99 I:  0 Tm:      0.00 NLPi:    3 Dpth:   17 Lvs:   71 Obj: -9.83E+01 Gap:  3.48E-02
Iter:   100 I:  0 Tm:      0.00 NLPi:    4 Dpth:   17 Lvs:   70 Obj: -9.39E+01 Gap:  3.48E-02
Iter:   101 I: -1 Tm:      0.00 NLPi:    1 Dpth:    7 Lvs:   69 Obj: -9.85E+01 Gap:  3.48E-02
Iter:   102 I: -1 Tm:      0.01 NLPi:    3 Dpth:    7 Lvs:   68 Obj: -9.85E+01 Gap:  3.48E-02
Iter:   103 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:   67 Obj: -9.39E+01 Gap:  3.48E-02
Iter:   104 I: -1 Tm:      0.00 NLPi:    1 Dpth:    6 Lvs:   66 Obj: -9.85E+01 Gap:  3.48E-02
Iter:   105 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:   68 Obj: -9.63E+01 Gap:  3.48E-02
Iter:   106 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:   70 Obj: -9.84E+01 Gap:  3.48E-02
Iter:   107 I: -1 Tm:      0.00 NLPi:    3 Dpth:    4 Lvs:  

--Integer Solution:  -9.55E+01 Lowest Leaf:  -9.80E+01 Gap:   2.54E-02
Iter:   188 I:  0 Tm:      0.00 NLPi:    2 Dpth:    6 Lvs:  111 Obj: -9.53E+01 Gap:  2.54E-02
--Integer Solution:  -9.55E+01 Lowest Leaf:  -9.80E+01 Gap:   2.54E-02
Iter:   189 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  110 Obj: -9.53E+01 Gap:  2.54E-02
Iter:   190 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  112 Obj: -9.70E+01 Gap:  2.54E-02
--Integer Solution:  -9.55E+01 Lowest Leaf:  -9.79E+01 Gap:   2.51E-02
Iter:   191 I:  0 Tm:      0.00 NLPi:    4 Dpth:    9 Lvs:  111 Obj: -9.04E+01 Gap:  2.51E-02
Iter:   192 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:  113 Obj: -9.71E+01 Gap:  2.51E-02
Iter:   193 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  115 Obj: -9.78E+01 Gap:  2.51E-02
Iter:   194 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  117 Obj: -9.67E+01 Gap:  2.51E-02
Iter:   195 I: -1 Tm:      0.00 NLPi:    1 Dpth:    7 Lvs:  116 Obj: -9.79E+01 Gap:  2.51E-02
Iter:   196 I: -1 Tm:      0.00 NLP

In [88]:
m.Equation(x2+x5+x11+x13+x29+x33<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           80
   Intermediates:            0
   Connections  :            0
   Equations    :           47
   Residuals    :           47
 
 Number of state variables:             80
 Number of total equations: -           46
 Number of slack variables: -           42
 ---------------------------------------
 Degrees of freedom       :             -8
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -9.94E+01 Gap:       NaN
Iter:     2 I: 

--Integer Solution:  -9.55E+01 Lowest Leaf:  -9.84E+01 Gap:   2.95E-02
Iter:   109 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:   78 Obj: -8.78E+01 Gap:  2.95E-02
Iter:   110 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:   80 Obj: -9.73E+01 Gap:  2.95E-02
Iter:   111 I: -1 Tm:      0.00 NLPi:    1 Dpth:    6 Lvs:   79 Obj: -9.83E+01 Gap:  2.95E-02
--Integer Solution:  -9.55E+01 Lowest Leaf:  -9.83E+01 Gap:   2.93E-02
Iter:   112 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   78 Obj: -9.31E+01 Gap:  2.93E-02
Iter:   113 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   79 Obj: -9.80E+01 Gap:  2.93E-02
--Integer Solution:  -9.55E+01 Lowest Leaf:  -9.83E+01 Gap:   2.91E-02
Iter:   114 I:  0 Tm:      0.00 NLPi:    2 Dpth:    4 Lvs:   78 Obj: -9.23E+01 Gap:  2.91E-02
Iter:   115 I:  0 Tm:      0.00 NLPi:    4 Dpth:    4 Lvs:   80 Obj: -9.74E+01 Gap:  2.91E-02
Iter:   116 I:  0 Tm:      0.00 NLPi:    4 Dpth:    4 Lvs:   82 Obj: -9.82E+01 Gap:  2.91E-02
--Integer Solution:  -9.55E+01 Lowe

Iter:   274 I: -1 Tm:      0.00 NLPi:    2 Dpth:    6 Lvs:  163 Obj: -9.75E+01 Gap:  2.07E-02
Iter:   275 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:  165 Obj: -9.63E+01 Gap:  2.07E-02
Iter:   276 I:  0 Tm:      0.00 NLPi:    5 Dpth:    6 Lvs:  167 Obj: -9.72E+01 Gap:  2.07E-02
Iter:   277 I: -1 Tm:      0.00 NLPi:    1 Dpth:    9 Lvs:  166 Obj: -9.75E+01 Gap:  2.07E-02
--Integer Solution:  -9.55E+01 Lowest Leaf:  -9.75E+01 Gap:   2.02E-02
Iter:   278 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:  165 Obj: -8.70E+01 Gap:  2.02E-02
Iter:   279 I:  0 Tm:      0.00 NLPi:    4 Dpth:    9 Lvs:  167 Obj: -9.74E+01 Gap:  2.02E-02
Iter:   280 I: -1 Tm:      0.00 NLPi:    1 Dpth:    7 Lvs:  166 Obj: -9.74E+01 Gap:  2.02E-02
--Integer Solution:  -9.55E+01 Lowest Leaf:  -9.74E+01 Gap:   2.02E-02
Iter:   281 I:  0 Tm:      0.00 NLPi:    2 Dpth:    7 Lvs:  165 Obj: -9.10E+01 Gap:  2.02E-02
Iter:   282 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:  167 Obj: -9.74E+01 Gap:  2.02E-02
--Integer So

In [89]:
m.Equation(x1+x7+x11+x14+x15+x35<= 5)
m.Equation(x35+x36==0)
m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           81
   Intermediates:            0
   Connections  :            0
   Equations    :           49
   Residuals    :           49
 
 Number of state variables:             81
 Number of total equations: -           48
 Number of slack variables: -           43
 ---------------------------------------
 Degrees of freedom       :            -10
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.01 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -9.87E+01 Gap:       NaN
Iter:     2 I: 

Iter:    91 I:  0 Tm:      0.01 NLPi:   14 Dpth:    8 Lvs:   56 Obj: -9.62E+01 Gap:  1.02E-02
Iter:    92 I:  0 Tm:      0.00 NLPi:    4 Dpth:    8 Lvs:   58 Obj: -9.63E+01 Gap:  1.02E-02
Iter:    93 I: -1 Tm:      0.00 NLPi:    2 Dpth:    8 Lvs:   57 Obj: -9.69E+01 Gap:  1.02E-02
--Integer Solution:  -9.59E+01 Lowest Leaf:  -9.69E+01 Gap:   1.02E-02
Iter:    94 I:  0 Tm:      0.00 NLPi:    4 Dpth:    8 Lvs:   56 Obj: -8.74E+01 Gap:  1.02E-02
Iter:    95 I:  0 Tm:      0.00 NLPi:    4 Dpth:    8 Lvs:   58 Obj: -9.67E+01 Gap:  1.02E-02
Iter:    96 I: -1 Tm:      0.00 NLPi:    1 Dpth:    7 Lvs:   57 Obj: -9.69E+01 Gap:  1.02E-02
Iter:    97 I:  0 Tm:      0.00 NLPi:    4 Dpth:    7 Lvs:   56 Obj: -9.41E+01 Gap:  1.02E-02
Iter:    98 I:  0 Tm:      0.00 NLPi:    4 Dpth:    7 Lvs:   55 Obj: -9.45E+01 Gap:  1.02E-02
Iter:    99 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:   54 Obj: -9.19E+01 Gap:  1.02E-02
Iter:   100 I:  0 Tm:      0.00 NLPi:    5 Dpth:    8 Lvs:   56 Obj: -9.66E+01 Gap:

In [90]:
m.Equation(x1+x7+x11+x13+x15+x30<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           82
   Intermediates:            0
   Connections  :            0
   Equations    :           50
   Residuals    :           50
 
 Number of state variables:             82
 Number of total equations: -           49
 Number of slack variables: -           44
 ---------------------------------------
 Degrees of freedom       :            -11
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.01 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -9.87E+01 Gap:       NaN
Iter:     2 I: 

Iter:    96 I:  0 Tm:      0.00 NLPi:    4 Dpth:    9 Lvs:   90 Obj: -9.65E+01 Gap:  3.18E-02
Iter:    97 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:   92 Obj: -9.66E+01 Gap:  3.18E-02
Iter:    98 I: -1 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:   91 Obj: -9.70E+01 Gap:  3.18E-02
--Integer Solution:  -9.39E+01 Lowest Leaf:  -9.70E+01 Gap:   3.14E-02
Iter:    99 I:  0 Tm:      0.00 NLPi:    2 Dpth:    7 Lvs:   90 Obj: -8.52E+01 Gap:  3.14E-02
Iter:   100 I:  0 Tm:      0.00 NLPi:    5 Dpth:    7 Lvs:   91 Obj: -9.66E+01 Gap:  3.14E-02
Iter:   101 I: -1 Tm:      0.00 NLPi:    3 Dpth:    5 Lvs:   90 Obj: -9.69E+01 Gap:  3.14E-02
--Integer Solution:  -9.39E+01 Lowest Leaf:  -9.69E+01 Gap:   3.14E-02
Iter:   102 I:  0 Tm:      0.00 NLPi:    4 Dpth:    5 Lvs:   89 Obj: -9.12E+01 Gap:  3.14E-02
Iter:   103 I:  0 Tm:      0.00 NLPi:    4 Dpth:    5 Lvs:   91 Obj: -9.65E+01 Gap:  3.14E-02
Iter:   104 I: -1 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:   90 Obj: -9.69E+01 Gap:  3.14E-02
Iter:   105 

In [91]:
m.Equation(x1+x5+x12+x15+x19+x29<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           83
   Intermediates:            0
   Connections  :            0
   Equations    :           51
   Residuals    :           51
 
 Number of state variables:             83
 Number of total equations: -           50
 Number of slack variables: -           45
 ---------------------------------------
 Degrees of freedom       :            -12
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    3 Dpth:    0 Lvs:    3 Obj: -9.87E+01 Gap:       NaN
Iter:     2 I: 

Iter:    83 I:  0 Tm:      0.00 NLPi:    4 Dpth:    7 Lvs:   58 Obj: -9.06E+01 Gap:  1.42E-02
Iter:    84 I:  0 Tm:      0.00 NLPi:    4 Dpth:    7 Lvs:   60 Obj: -9.70E+01 Gap:  1.42E-02
--Integer Solution:  -9.57E+01 Lowest Leaf:  -9.71E+01 Gap:   1.41E-02
Iter:    85 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:   59 Obj: -8.24E+01 Gap:  1.41E-02
Iter:    86 I:  0 Tm:      0.00 NLPi:    4 Dpth:   10 Lvs:   61 Obj: -9.68E+01 Gap:  1.41E-02
Iter:    87 I: -1 Tm:      0.00 NLPi:    1 Dpth:    5 Lvs:   60 Obj: -9.71E+01 Gap:  1.41E-02
Iter:    88 I:  0 Tm:      0.00 NLPi:    3 Dpth:    5 Lvs:   62 Obj: -9.69E+01 Gap:  1.41E-02
Iter:    89 I:  0 Tm:      0.00 NLPi:    5 Dpth:    5 Lvs:   64 Obj: -9.67E+01 Gap:  1.41E-02
Iter:    90 I: -1 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   63 Obj: -9.71E+01 Gap:  1.41E-02
Iter:    91 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:   62 Obj: -9.56E+01 Gap:  1.41E-02
Iter:    92 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:   63 Obj: -9.68E+01 Gap:

In [92]:
m.Equation(x1+x7+x12+x13+x15+x33<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           84
   Intermediates:            0
   Connections  :            0
   Equations    :           52
   Residuals    :           52
 
 Number of state variables:             84
 Number of total equations: -           51
 Number of slack variables: -           46
 ---------------------------------------
 Degrees of freedom       :            -13
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -9.87E+01 Gap:       NaN
Iter:     2 I: 

Iter:    92 I:  0 Tm:      0.01 NLPi:   13 Dpth:    4 Lvs:   69 Obj: -9.71E+01 Gap:  1.49E-02
Iter:    93 I: -1 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   68 Obj: -9.72E+01 Gap:  1.49E-02
Iter:    94 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   70 Obj: -9.64E+01 Gap:  1.49E-02
Iter:    95 I:  0 Tm:      0.01 NLPi:    6 Dpth:    6 Lvs:   72 Obj: -9.63E+01 Gap:  1.49E-02
Iter:    96 I: -1 Tm:      0.00 NLPi:    1 Dpth:    6 Lvs:   71 Obj: -9.72E+01 Gap:  1.49E-02
Iter:    97 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:   70 Obj: -9.50E+01 Gap:  1.49E-02
Iter:    98 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   72 Obj: -9.71E+01 Gap:  1.49E-02
Iter:    99 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   74 Obj: -9.67E+01 Gap:  1.49E-02
Iter:   100 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   76 Obj: -9.65E+01 Gap:  1.49E-02
Iter:   101 I: -1 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:   75 Obj: -9.71E+01 Gap:  1.49E-02
Iter:   102 I:  0 Tm:      0.00 NLPi:    4 Dpth:    8 Lvs:  

x1: 1.0 Tom Brady
x2: 0.0 Tom Brady /King
x3: 1.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 0.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 0.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 0.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 0.0 Antonio Brown
x12: 1.0 Antonio Brown /King
x13: 1.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 0.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 0.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 0.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 0.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 1.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x31: 0.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 1.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 0.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


In [93]:
m.Equation(x1+x3+x12+x13+x29+x33<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           85
   Intermediates:            0
   Connections  :            0
   Equations    :           53
   Residuals    :           53
 
 Number of state variables:             85
 Number of total equations: -           52
 Number of slack variables: -           47
 ---------------------------------------
 Degrees of freedom       :            -14
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.01 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -9.87E+01 Gap:       NaN
Iter:     2 I: 

Iter:    87 I: -1 Tm:      0.00 NLPi:    2 Dpth:    4 Lvs:   60 Obj: -9.71E+01 Gap:  1.28E-02
Iter:    88 I:  0 Tm:      0.00 NLPi:    3 Dpth:    4 Lvs:   62 Obj: -9.70E+01 Gap:  1.28E-02
Iter:    89 I:  0 Tm:      0.00 NLPi:    3 Dpth:    4 Lvs:   64 Obj: -9.67E+01 Gap:  1.28E-02
--Integer Solution:  -9.61E+01 Lowest Leaf:  -9.71E+01 Gap:   1.05E-02
Iter:    90 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:   63 Obj: -8.83E+01 Gap:  1.05E-02
Iter:    91 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:   65 Obj: -9.70E+01 Gap:  1.05E-02
Iter:    92 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   67 Obj: -9.66E+01 Gap:  1.05E-02
Iter:    93 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   66 Obj: -9.58E+01 Gap:  1.05E-02
Iter:    94 I: -1 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   65 Obj: -9.71E+01 Gap:  1.05E-02
Iter:    95 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   64 Obj: -9.61E+01 Gap:  1.05E-02
Iter:    96 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   66 Obj: -9.65E+01 Gap:

In [94]:
m.Equation(x2+x3+x11+x15+x29+x33<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           86
   Intermediates:            0
   Connections  :            0
   Equations    :           54
   Residuals    :           54
 
 Number of state variables:             86
 Number of total equations: -           53
 Number of slack variables: -           48
 ---------------------------------------
 Degrees of freedom       :            -15
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.01 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -9.87E+01 Gap:       NaN
Iter:     2 I: 

Iter:    86 I:  0 Tm:      0.00 NLPi:    5 Dpth:    5 Lvs:   84 Obj: -9.63E+01 Gap:  3.35E-02
Iter:    87 I: -1 Tm:      0.00 NLPi:    1 Dpth:    6 Lvs:   83 Obj: -9.71E+01 Gap:  3.35E-02
Iter:    88 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   85 Obj: -9.69E+01 Gap:  3.35E-02
Iter:    89 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:   87 Obj: -9.65E+01 Gap:  3.35E-02
Iter:    90 I: -9 Tm:      0.19 NLPi:  251 Dpth:    4 Lvs:   86 Obj: -9.70E+01 Gap:  3.35E-02
Iter:    91 I:  0 Tm:      0.00 NLPi:    3 Dpth:    4 Lvs:   88 Obj: -9.64E+01 Gap:  3.35E-02
Iter:    92 I:  0 Tm:      0.00 NLPi:    3 Dpth:    4 Lvs:   90 Obj: -9.67E+01 Gap:  3.35E-02
Iter:    93 I: -1 Tm:      0.00 NLPi:    1 Dpth:    7 Lvs:   89 Obj: -9.70E+01 Gap:  3.35E-02
Iter:    94 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:   88 Obj: -9.39E+01 Gap:  3.35E-02
Iter:    95 I:  0 Tm:      0.00 NLPi:    4 Dpth:    7 Lvs:   90 Obj: -9.59E+01 Gap:  3.35E-02
Iter:    96 I: -1 Tm:      0.00 NLPi:    1 Dpth:    5 Lvs:  

Iter:   237 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  160 Obj: -9.56E+01 Gap:  1.39E-02
Iter:   238 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:  162 Obj: -9.60E+01 Gap:  1.39E-02
Iter:   239 I: -1 Tm:      0.00 NLPi:    3 Dpth:   11 Lvs:  161 Obj: -9.62E+01 Gap:  1.39E-02
Iter:   240 I:  0 Tm:      0.00 NLPi:    4 Dpth:   11 Lvs:  160 Obj: -9.47E+01 Gap:  1.39E-02
Iter:   241 I:  0 Tm:      0.00 NLPi:    4 Dpth:   11 Lvs:  162 Obj: -9.60E+01 Gap:  1.39E-02
Iter:   242 I: -1 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:  161 Obj: -9.62E+01 Gap:  1.39E-02
Iter:   243 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:  162 Obj: -9.62E+01 Gap:  1.39E-02
Iter:   244 I:  0 Tm:      0.00 NLPi:    4 Dpth:    7 Lvs:  161 Obj: -9.45E+01 Gap:  1.39E-02
Iter:   245 I: -1 Tm:      0.00 NLPi:    1 Dpth:    4 Lvs:  160 Obj: -9.62E+01 Gap:  1.39E-02
Iter:   246 I:  0 Tm:      0.00 NLPi:    3 Dpth:    4 Lvs:  162 Obj: -9.62E+01 Gap:  1.39E-02
Iter:   247 I:  0 Tm:      0.00 NLPi:    4 Dpth:    4 Lvs:  

Iter:   365 I:  0 Tm:      0.00 NLPi:    4 Dpth:    8 Lvs:  175 Obj: -9.58E+01 Gap:  1.05E-02
--Integer Solution:  -9.49E+01 Lowest Leaf:  -9.59E+01 Gap:   1.04E-02
Iter:   366 I:  0 Tm:      0.00 NLPi:    4 Dpth:    8 Lvs:  174 Obj: -8.05E+01 Gap:  1.04E-02
Iter:   367 I:  0 Tm:      0.00 NLPi:    4 Dpth:    8 Lvs:  176 Obj: -9.58E+01 Gap:  1.04E-02
Iter:   368 I: -1 Tm:      0.00 NLPi:    3 Dpth:    5 Lvs:  175 Obj: -9.59E+01 Gap:  1.04E-02
--Integer Solution:  -9.49E+01 Lowest Leaf:  -9.59E+01 Gap:   1.04E-02
Iter:   369 I:  0 Tm:      0.00 NLPi:    4 Dpth:    5 Lvs:  174 Obj: -8.93E+01 Gap:  1.04E-02
Iter:   370 I:  0 Tm:      0.00 NLPi:    4 Dpth:    5 Lvs:  176 Obj: -9.57E+01 Gap:  1.04E-02
--Integer Solution:  -9.58E+01 Lowest Leaf:  -9.59E+01 Gap:   1.28E-03
Iter:   371 I:  0 Tm:      0.00 NLPi:    2 Dpth:    7 Lvs:  176 Obj: -9.58E+01 Gap:  1.28E-03
 Successful solution
 
 ---------------------------------------------------
 Solver         :  APOPT (v1.0)
 Solution time  :    

In [95]:
m.Equation(x1+x5+x11+x13+x16+x33<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           87
   Intermediates:            0
   Connections  :            0
   Equations    :           55
   Residuals    :           55
 
 Number of state variables:             87
 Number of total equations: -           54
 Number of slack variables: -           49
 ---------------------------------------
 Degrees of freedom       :            -16
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.01 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -9.87E+01 Gap:       NaN
Iter:     2 I: 

Iter:    88 I:  0 Tm:      0.00 NLPi:    4 Dpth:    7 Lvs:   80 Obj: -9.68E+01 Gap:  3.35E-02
Iter:    89 I: -1 Tm:      0.00 NLPi:    2 Dpth:    7 Lvs:   79 Obj: -9.69E+01 Gap:  3.35E-02
--Integer Solution:  -9.39E+01 Lowest Leaf:  -9.69E+01 Gap:   3.12E-02
Iter:    90 I:  0 Tm:      0.00 NLPi:    4 Dpth:    7 Lvs:   78 Obj: -8.74E+01 Gap:  3.12E-02
Iter:    91 I:  0 Tm:      0.00 NLPi:    4 Dpth:    7 Lvs:   80 Obj: -9.67E+01 Gap:  3.12E-02
Iter:    92 I: -1 Tm:      0.00 NLPi:    1 Dpth:    6 Lvs:   79 Obj: -9.69E+01 Gap:  3.12E-02
--Integer Solution:  -9.39E+01 Lowest Leaf:  -9.69E+01 Gap:   3.12E-02
Iter:    93 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   78 Obj: -8.40E+01 Gap:  3.12E-02
Iter:    94 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   80 Obj: -9.68E+01 Gap:  3.12E-02
Iter:    95 I: -1 Tm:      0.00 NLPi:    1 Dpth:    8 Lvs:   79 Obj: -9.69E+01 Gap:  3.12E-02
Iter:    96 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:   81 Obj: -9.69E+01 Gap:  3.12E-02
Iter:    97 

x1: 1.0 Tom Brady
x2: 0.0 Tom Brady /King
x3: 1.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 0.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 0.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 0.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 1.0 Antonio Brown
x12: 0.0 Antonio Brown /King
x13: 1.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 0.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 0.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 1.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 0.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 0.0 Zach Ertz
x30: 1.0 Zach Ertz /King
x31: 0.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 0.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 0.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


Devonta Smith

In [96]:
m.Equation(x1+x3+x11+x13+x19+x30<= 5)

m.Equation(x9+x10==1)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           88
   Intermediates:            0
   Connections  :            0
   Equations    :           57
   Residuals    :           57
 
 Number of state variables:             88
 Number of total equations: -           56
 Number of slack variables: -           50
 ---------------------------------------
 Degrees of freedom       :            -18
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -9.65E+01 Gap:       NaN
Iter:     2 I: 

x1: 1.0 Tom Brady
x2: 0.0 Tom Brady /King
x3: 0.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 0.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 0.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 1.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 0.0 Antonio Brown
x12: 1.0 Antonio Brown /King
x13: 1.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 0.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 0.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 1.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 0.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 1.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x31: 0.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 0.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 0.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


In [97]:
m.Equation(x1+x9+x12+x13+x19+x29<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           89
   Intermediates:            0
   Connections  :            0
   Equations    :           58
   Residuals    :           58
 
 Number of state variables:             89
 Number of total equations: -           57
 Number of slack variables: -           51
 ---------------------------------------
 Degrees of freedom       :            -19
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.01 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -9.65E+01 Gap:       NaN
Iter:     2 I: 

Iter:    82 I:  0 Tm:      0.00 NLPi:    4 Dpth:    5 Lvs:   39 Obj: -9.30E+01 Gap:  1.04E-02
Iter:    83 I:  0 Tm:      0.00 NLPi:    4 Dpth:    5 Lvs:   41 Obj: -9.45E+01 Gap:  1.04E-02
--Integer Solution:  -9.42E+01 Lowest Leaf:  -9.46E+01 Gap:   4.77E-03
Iter:    84 I:  0 Tm:      0.00 NLPi:    2 Dpth:    4 Lvs:   41 Obj: -9.27E+01 Gap:  4.77E-03
 Successful solution
 
 ---------------------------------------------------
 Solver         :  APOPT (v1.0)
 Solution time  :   0.323300000000017      sec
 Objective      :   -94.1500000000000     
 Successful solution
 ---------------------------------------------------
 
x1: 1.0 Tom Brady
x2: 0.0 Tom Brady /King
x3: 0.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 0.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 0.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 1.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 0.0 Antonio Brown
x12: 1.0 Antonio Brown /King
x13: 1.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 1.0 Miles Sanders
x16: 0.

In [98]:
m.Equation(x1+x9+x12+x13+x15+x33<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           90
   Intermediates:            0
   Connections  :            0
   Equations    :           59
   Residuals    :           59
 
 Number of state variables:             90
 Number of total equations: -           58
 Number of slack variables: -           52
 ---------------------------------------
 Degrees of freedom       :            -20
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -9.65E+01 Gap:       NaN
Iter:     2 I: 

Iter:    82 I:  0 Tm:      0.00 NLPi:    3 Dpth:    4 Lvs:   54 Obj: -9.48E+01 Gap:  1.75E-02
Iter:    83 I:  0 Tm:      0.00 NLPi:    3 Dpth:    4 Lvs:   56 Obj: -9.42E+01 Gap:  1.75E-02
Iter:    84 I: -1 Tm:      0.00 NLPi:    2 Dpth:    9 Lvs:   55 Obj: -9.50E+01 Gap:  1.75E-02
Iter:    85 I:  0 Tm:      0.00 NLPi:    2 Dpth:    9 Lvs:   57 Obj: -9.48E+01 Gap:  1.75E-02
Iter:    86 I:  0 Tm:      0.00 NLPi:    4 Dpth:    9 Lvs:   56 Obj: -9.25E+01 Gap:  1.75E-02
Iter:    87 I: -1 Tm:      0.00 NLPi:    1 Dpth:    7 Lvs:   55 Obj: -9.49E+01 Gap:  1.75E-02
Iter:    88 I:  0 Tm:      0.00 NLPi:    4 Dpth:    7 Lvs:   54 Obj: -9.26E+01 Gap:  1.75E-02
Iter:    89 I:  0 Tm:      0.00 NLPi:    4 Dpth:    7 Lvs:   56 Obj: -9.46E+01 Gap:  1.75E-02
--Integer Solution:  -9.35E+01 Lowest Leaf:  -9.49E+01 Gap:   1.41E-02
Iter:    90 I:  0 Tm:      0.00 NLPi:    2 Dpth:    7 Lvs:   55 Obj: -9.35E+01 Gap:  1.41E-02
--Integer Solution:  -9.35E+01 Lowest Leaf:  -9.49E+01 Gap:   1.41E-02
Iter:    91 

In [99]:
m.Equation(x1+x9+x11+x13+x15+x30<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           91
   Intermediates:            0
   Connections  :            0
   Equations    :           60
   Residuals    :           60
 
 Number of state variables:             91
 Number of total equations: -           59
 Number of slack variables: -           53
 ---------------------------------------
 Degrees of freedom       :            -21
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.01 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -9.65E+01 Gap:       NaN
Iter:     2 I: 

Iter:    78 I:  0 Tm:      0.00 NLPi:    5 Dpth:    9 Lvs:   51 Obj: -9.13E+01 Gap:  2.48E-02
Iter:    79 I: -1 Tm:      0.00 NLPi:    3 Dpth:    5 Lvs:   50 Obj: -9.49E+01 Gap:  2.48E-02
Iter:    80 I:  0 Tm:      0.00 NLPi:    5 Dpth:    5 Lvs:   52 Obj: -9.48E+01 Gap:  2.48E-02
Iter:    81 I:  0 Tm:      0.00 NLPi:    3 Dpth:    5 Lvs:   54 Obj: -9.41E+01 Gap:  2.48E-02
Iter:    82 I: -1 Tm:      0.00 NLPi:    2 Dpth:    6 Lvs:   53 Obj: -9.48E+01 Gap:  2.48E-02
Iter:    83 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:   55 Obj: -9.46E+01 Gap:  2.48E-02
Iter:    84 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   57 Obj: -9.48E+01 Gap:  2.48E-02
--Integer Solution:  -9.27E+01 Lowest Leaf:  -9.48E+01 Gap:   2.23E-02
Iter:    85 I:  0 Tm:      0.00 NLPi:    2 Dpth:    7 Lvs:   56 Obj: -9.08E+01 Gap:  2.23E-02
Iter:    86 I:  0 Tm:      0.00 NLPi:    4 Dpth:    7 Lvs:   58 Obj: -9.38E+01 Gap:  2.23E-02
Iter:    87 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:   59 Obj: -9.48E+01 Gap:

In [100]:
m.Equation(x1+x9+x11+x14+x19+x29<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           92
   Intermediates:            0
   Connections  :            0
   Equations    :           61
   Residuals    :           61
 
 Number of state variables:             92
 Number of total equations: -           60
 Number of slack variables: -           54
 ---------------------------------------
 Degrees of freedom       :            -22
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.01 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -9.65E+01 Gap:       NaN
Iter:     2 I: 

--Integer Solution:  -9.27E+01 Lowest Leaf:  -9.48E+01 Gap:   2.21E-02
Iter:    81 I:  0 Tm:      0.00 NLPi:    2 Dpth:    5 Lvs:   63 Obj: -8.01E+01 Gap:  2.21E-02
Iter:    82 I:  0 Tm:      0.00 NLPi:    4 Dpth:    5 Lvs:   64 Obj: -9.45E+01 Gap:  2.21E-02
--Integer Solution:  -9.27E+01 Lowest Leaf:  -9.48E+01 Gap:   2.20E-02
Iter:    83 I:  0 Tm:      0.00 NLPi:    2 Dpth:    6 Lvs:   63 Obj: -9.27E+01 Gap:  2.20E-02
Iter:    84 I:  0 Tm:      0.00 NLPi:    4 Dpth:    6 Lvs:   62 Obj: -9.12E+01 Gap:  2.20E-02
Iter:    85 I:  0 Tm:      0.00 NLPi:    3 Dpth:    6 Lvs:   63 Obj: -9.46E+01 Gap:  2.20E-02
Iter:    86 I: -1 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:   62 Obj: -9.47E+01 Gap:  2.20E-02
Iter:    87 I:  0 Tm:      0.01 NLPi:   13 Dpth:    8 Lvs:   64 Obj: -9.38E+01 Gap:  2.20E-02
Iter:    88 I:  0 Tm:      0.00 NLPi:    4 Dpth:    8 Lvs:   66 Obj: -9.46E+01 Gap:  2.20E-02
Iter:    89 I: -1 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:   65 Obj: -9.46E+01 Gap:  2.20E-02
Iter:    90 

x1: 1.0 Tom Brady
x2: 0.0 Tom Brady /King
x3: 0.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 0.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 0.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 1.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 1.0 Antonio Brown
x12: 0.0 Antonio Brown /King
x13: 0.0 Leonard Fournette
x14: 1.0 Leonard Fournette /King
x15: 1.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 0.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 0.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 0.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 0.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x31: 0.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 1.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 0.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


In [101]:
m.Equation(x1+x9+x11+x14+x15+x33<= 5)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           93
   Intermediates:            0
   Connections  :            0
   Equations    :           62
   Residuals    :           62
 
 Number of state variables:             93
 Number of total equations: -           61
 Number of slack variables: -           55
 ---------------------------------------
 Degrees of freedom       :            -23
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.01 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -9.65E+01 Gap:       NaN
Iter:     2 I: 

Iter:    77 I: -1 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:   52 Obj: -9.46E+01 Gap:  2.79E-02
Iter:    78 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:   51 Obj: -9.25E+01 Gap:  2.79E-02
Iter:    79 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:   53 Obj: -9.45E+01 Gap:  2.79E-02
--Integer Solution:  -9.27E+01 Lowest Leaf:  -9.46E+01 Gap:   2.03E-02
Iter:    80 I:  0 Tm:      0.00 NLPi:    2 Dpth:    9 Lvs:   52 Obj: -8.30E+01 Gap:  2.03E-02
Iter:    81 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:   54 Obj: -9.31E+01 Gap:  2.03E-02
Iter:    82 I:  0 Tm:      0.00 NLPi:    4 Dpth:    9 Lvs:   56 Obj: -9.41E+01 Gap:  2.03E-02
Iter:    83 I: -1 Tm:      0.00 NLPi:    2 Dpth:    3 Lvs:   55 Obj: -9.46E+01 Gap:  2.03E-02
Iter:    84 I:  0 Tm:      0.00 NLPi:    3 Dpth:    3 Lvs:   57 Obj: -9.35E+01 Gap:  2.03E-02
Iter:    85 I:  0 Tm:      0.00 NLPi:    3 Dpth:    3 Lvs:   59 Obj: -9.41E+01 Gap:  2.03E-02
--Integer Solution:  -9.27E+01 Lowest Leaf:  -9.46E+01 Gap:   2.02E-02
Iter:    86 

x1: 1.0 Tom Brady
x2: 0.0 Tom Brady /King
x3: 1.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 0.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 0.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 1.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 0.0 Antonio Brown
x12: 1.0 Antonio Brown /King
x13: 0.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 0.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 0.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 0.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 0.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 1.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x31: 0.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 1.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 0.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


remove D smith mandate and insert gio

In [108]:
m.Equation(x1+x3+x9+x12+x29+x33<= 5)
m.Equation(x17+x18==1)
m.Equation(x31+x32==0)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model5 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           60
   Intermediates:            0
   Connections  :            0
   Equations    :           29
   Residuals    :           29
 
 Number of state variables:             60
 Number of total equations: -           28
 Number of slack variables: -           22
 ---------------------------------------
 Degrees of freedom       :             10
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -9.44E+01 Gap:       NaN
--Integer Solution:  -9.39E+01 Lowes

Gio

In [110]:
m.Equation(x2+x11+x13+x15+x17+x29<= 5)


m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model5 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           62
   Intermediates:            0
   Connections  :            0
   Equations    :           31
   Residuals    :           31
 
 Number of state variables:             62
 Number of total equations: -           30
 Number of slack variables: -           24
 ---------------------------------------
 Degrees of freedom       :              8
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -9.44E+01 Gap:       NaN
--Integer Solution:  -9.11E+01 Lowes

x1: 1.0 Tom Brady
x2: 0.0 Tom Brady /King
x3: 0.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 1.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 0.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 0.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 0.0 Antonio Brown
x12: 1.0 Antonio Brown /King
x13: 1.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 0.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 1.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 0.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 0.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 0.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x31: 0.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 0.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 1.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


In [111]:
m.Equation(x1+x3+x9+x12+x29+x33<= 5)
m.Equation(x17+x18==1)
m.Equation(x31+x32==0)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model5 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           63
   Intermediates:            0
   Connections  :            0
   Equations    :           34
   Residuals    :           34
 
 Number of state variables:             63
 Number of total equations: -           33
 Number of slack variables: -           25
 ---------------------------------------
 Degrees of freedom       :              5
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -9.44E+01 Gap:       NaN
--Integer Solution:  -9.11E+01 Lowes

x1: 1.0 Tom Brady
x2: 0.0 Tom Brady /King
x3: 0.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 1.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 0.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 0.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 0.0 Antonio Brown
x12: 1.0 Antonio Brown /King
x13: 1.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 0.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 1.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 0.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 0.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 0.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x31: 0.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 0.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 1.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


In [112]:
m.Equation(x1+x5+x12+x13+x17+x35<= 5)
m.Equation(x35+x36==0)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model5 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           64
   Intermediates:            0
   Connections  :            0
   Equations    :           36
   Residuals    :           36
 
 Number of state variables:             64
 Number of total equations: -           35
 Number of slack variables: -           26
 ---------------------------------------
 Degrees of freedom       :              3
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -9.43E+01 Gap:       NaN
Iter:     2 I: -1 Tm:      0.00 NLPi

x1: 1.0 Tom Brady
x2: 0.0 Tom Brady /King
x3: 0.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 1.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 0.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 0.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 0.0 Antonio Brown
x12: 1.0 Antonio Brown /King
x13: 0.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 1.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 1.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 0.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 0.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 1.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x31: 0.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 0.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 0.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


In [113]:
m.Equation(x1+x5+x12+x15+x17+x29<= 5)


m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model5 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           65
   Intermediates:            0
   Connections  :            0
   Equations    :           37
   Residuals    :           37
 
 Number of state variables:             65
 Number of total equations: -           36
 Number of slack variables: -           27
 ---------------------------------------
 Degrees of freedom       :              2
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.01 NLPi:   10 Dpth:    0 Lvs:    3 Obj: -9.43E+01 Gap:       NaN
Iter:     2 I: -1 Tm:      0.00 NLPi

Iter:   130 I: -1 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:   59 Obj: -9.24E+01 Gap:  1.04E-02
--Integer Solution:  -9.14E+01 Lowest Leaf:  -9.24E+01 Gap:   1.01E-02
Iter:   131 I:  0 Tm:      0.00 NLPi:    4 Dpth:    9 Lvs:   58 Obj: -7.59E+01 Gap:  1.01E-02
Iter:   132 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:   59 Obj: -9.24E+01 Gap:  1.01E-02
Iter:   133 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:   61 Obj: -9.24E+01 Gap:  1.01E-02
Iter:   134 I:  0 Tm:      0.00 NLPi:    4 Dpth:   10 Lvs:   60 Obj: -8.97E+01 Gap:  1.01E-02
--Integer Solution:  -9.14E+01 Lowest Leaf:  -9.24E+01 Gap:   9.94E-03
Iter:   135 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:   60 Obj: -8.15E+01 Gap:  9.94E-03
 Successful solution
 
 ---------------------------------------------------
 Solver         :  APOPT (v1.0)
 Solution time  :   0.347499999999854      sec
 Objective      :   -91.4500000000000     
 Successful solution
 ---------------------------------------------------
 
x1: 1.0 Tom Brady
x2

In [114]:
m.Equation(x1+x12+x13+x15+x17+x19<= 5)


m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model5 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           66
   Intermediates:            0
   Connections  :            0
   Equations    :           38
   Residuals    :           38
 
 Number of state variables:             66
 Number of total equations: -           37
 Number of slack variables: -           28
 ---------------------------------------
 Degrees of freedom       :              1
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -9.43E+01 Gap:       NaN
Iter:     2 I: -1 Tm:      0.00 NLPi

Iter:   139 I: -1 Tm:      0.00 NLPi:    2 Dpth:    8 Lvs:   72 Obj: -9.24E+01 Gap:  1.18E-02
Iter:   140 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:   74 Obj: -9.17E+01 Gap:  1.18E-02
Iter:   141 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:   73 Obj: -9.10E+01 Gap:  1.18E-02
Iter:   142 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:   72 Obj: -9.12E+01 Gap:  1.18E-02
Iter:   143 I:  0 Tm:      0.00 NLPi:    3 Dpth:   10 Lvs:   74 Obj: -9.20E+01 Gap:  1.18E-02
Iter:   144 I: -1 Tm:      0.00 NLPi:    2 Dpth:    7 Lvs:   73 Obj: -9.24E+01 Gap:  1.18E-02
Iter:   145 I:  0 Tm:      0.01 NLPi:   13 Dpth:    7 Lvs:   75 Obj: -9.24E+01 Gap:  1.18E-02
--Integer Solution:  -9.13E+01 Lowest Leaf:  -9.24E+01 Gap:   1.09E-02
Iter:   146 I:  0 Tm:      0.00 NLPi:    2 Dpth:    7 Lvs:   74 Obj: -8.20E+01 Gap:  1.09E-02
Iter:   147 I: -1 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:   73 Obj: -9.24E+01 Gap:  1.09E-02
--Integer Solution:  -9.13E+01 Lowest Leaf:  -9.24E+01 Gap:   1.09E-02
Iter:   148 

In [115]:
m.Equation(x1+x5+x12+x13+x17+x33<= 5)


m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model5 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           67
   Intermediates:            0
   Connections  :            0
   Equations    :           39
   Residuals    :           39
 
 Number of state variables:             67
 Number of total equations: -           38
 Number of slack variables: -           29
 ---------------------------------------
 Degrees of freedom       :              0
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -9.43E+01 Gap:       NaN
Iter:     2 I: -1 Tm:      0.00 NLPi

Iter:   130 I:  0 Tm:      0.00 NLPi:    3 Dpth:    9 Lvs:   71 Obj: -9.25E+01 Gap:  1.26E-02
Iter:   131 I:  0 Tm:      0.00 NLPi:    4 Dpth:    9 Lvs:   70 Obj: -9.08E+01 Gap:  1.26E-02
--Integer Solution:  -9.13E+01 Lowest Leaf:  -9.25E+01 Gap:   1.25E-02
Iter:   132 I:  0 Tm:      0.00 NLPi:    2 Dpth:    7 Lvs:   69 Obj: -9.11E+01 Gap:  1.25E-02
--Integer Solution:  -9.13E+01 Lowest Leaf:  -9.25E+01 Gap:   1.25E-02
Iter:   133 I:  0 Tm:      0.00 NLPi:    2 Dpth:    7 Lvs:   68 Obj: -9.11E+01 Gap:  1.25E-02
Iter:   134 I:  0 Tm:      0.00 NLPi:    3 Dpth:    7 Lvs:   69 Obj: -9.25E+01 Gap:  1.25E-02
--Integer Solution:  -9.13E+01 Lowest Leaf:  -9.25E+01 Gap:   1.23E-02
Iter:   135 I:  0 Tm:      0.00 NLPi:    2 Dpth:    8 Lvs:   68 Obj: -8.44E+01 Gap:  1.23E-02
Iter:   136 I:  0 Tm:      0.00 NLPi:    3 Dpth:    8 Lvs:   69 Obj: -9.25E+01 Gap:  1.23E-02
Iter:   137 I: -1 Tm:      0.00 NLPi:    2 Dpth:   10 Lvs:   68 Obj: -9.25E+01 Gap:  1.23E-02
--Integer Solution:  -9.13E+01 Lowe

x1: 1.0 Tom Brady
x2: 0.0 Tom Brady /King
x3: 0.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 0.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 0.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 1.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 0.0 Antonio Brown
x12: 1.0 Antonio Brown /King
x13: 1.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 0.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 1.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 0.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 0.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 1.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x31: 0.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 0.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 0.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


quez watkins

In [117]:
m.Equation(x1+x5+x12+x13+x17+x33<= 5)
m.Equation(x31+x32==0)
m.Equation(x23+x24==1)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model6 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           59
   Intermediates:            0
   Connections  :            0
   Equations    :           27
   Residuals    :           27
 
 Number of state variables:             59
 Number of total equations: -           26
 Number of slack variables: -           21
 ---------------------------------------
 Degrees of freedom       :             12
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -9.62E+01 Gap:       NaN
--Integer Solution:  -9.49E+01 Lowes

In [118]:
m.Equation(x2+x11+x13+x15+x23+x29<= 5)


m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model6 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           60
   Intermediates:            0
   Connections  :            0
   Equations    :           28
   Residuals    :           28
 
 Number of state variables:             60
 Number of total equations: -           27
 Number of slack variables: -           22
 ---------------------------------------
 Degrees of freedom       :             11
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -9.62E+01 Gap:       NaN
--Integer Solution:  -9.21E+01 Lowes

In [119]:
m.Equation(x2+x7+x11+x13+x23+x35<= 5)
m.Equation(x35+x36==0)

m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model6 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           61
   Intermediates:            0
   Connections  :            0
   Equations    :           30
   Residuals    :           30
 
 Number of state variables:             61
 Number of total equations: -           29
 Number of slack variables: -           23
 ---------------------------------------
 Degrees of freedom       :              9
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -9.61E+01 Gap:       NaN
Iter:     2 I: -1 Tm:      0.00 NLPi

In [120]:
m.Equation(x1+x7+x12+x13+x23+x29<= 5)


m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model6 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           62
   Intermediates:            0
   Connections  :            0
   Equations    :           31
   Residuals    :           31
 
 Number of state variables:             62
 Number of total equations: -           30
 Number of slack variables: -           24
 ---------------------------------------
 Degrees of freedom       :              8
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -9.61E+01 Gap:       NaN
Iter:     2 I: -1 Tm:      0.00 NLPi

x1: 1.0 Tom Brady
x2: 0.0 Tom Brady /King
x3: 0.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 1.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 0.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 0.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 0.0 Antonio Brown
x12: 1.0 Antonio Brown /King
x13: 0.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 1.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 0.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 0.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 1.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 1.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x31: 0.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 0.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 0.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


In [121]:
m.Equation(x1+x5+x12+x15+x23+x29<= 5)


m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model6 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           63
   Intermediates:            0
   Connections  :            0
   Equations    :           32
   Residuals    :           32
 
 Number of state variables:             63
 Number of total equations: -           31
 Number of slack variables: -           25
 ---------------------------------------
 Degrees of freedom       :              7
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -9.61E+01 Gap:       NaN
Iter:     2 I: -1 Tm:      0.00 NLPi

x1: 1.0 Tom Brady
x2: 0.0 Tom Brady /King
x3: 1.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 0.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 0.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 0.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 1.0 Antonio Brown
x12: 0.0 Antonio Brown /King
x13: 1.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 0.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 0.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 0.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 1.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 0.0 Zach Ertz
x30: 1.0 Zach Ertz /King
x31: 0.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 0.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 0.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


In [122]:
m.Equation(x1+x3+x11+x13+x23+x29<= 5)


m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model6 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           64
   Intermediates:            0
   Connections  :            0
   Equations    :           33
   Residuals    :           33
 
 Number of state variables:             64
 Number of total equations: -           32
 Number of slack variables: -           26
 ---------------------------------------
 Degrees of freedom       :              6
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    3 Obj: -9.61E+01 Gap:       NaN
Iter:     2 I: -1 Tm:      0.00 NLPi

x1: 1.0 Tom Brady
x2: 0.0 Tom Brady /King
x3: 0.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 0.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 1.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 0.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 1.0 Antonio Brown
x12: 0.0 Antonio Brown /King
x13: 0.0 Leonard Fournette
x14: 1.0 Leonard Fournette /King
x15: 0.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 0.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 0.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 1.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 1.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x31: 0.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 0.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 0.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


In [123]:
m.Equation(x1+x7+x11+x14+x23+x29<= 5)


m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model6 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           65
   Intermediates:            0
   Connections  :            0
   Equations    :           34
   Residuals    :           34
 
 Number of state variables:             65
 Number of total equations: -           33
 Number of slack variables: -           27
 ---------------------------------------
 Degrees of freedom       :              5
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -9.61E+01 Gap:       NaN
Iter:     2 I: -1 Tm:      0.00 NLPi

x1: 0.0 Tom Brady
x2: 1.0 Tom Brady /King
x3: 0.0 Jalen Hurts
x4: 0.0 Jalen Hurts /King
x5: 0.0 Mike Evans
x6: 0.0 Mike Evans /King
x7: 1.0 Chris Godwin
x8: 0.0 Chris Godwin /King
x9: 0.0 DeVonta Smith
x10: 0.0 DeVonta Smith /King
x11: 1.0 Antonio Brown
x12: 0.0 Antonio Brown /King
x13: 1.0 Leonard Fournette
x14: 0.0 Leonard Fournette /King
x15: 0.0 Miles Sanders
x16: 0.0 Miles Sanders /King
x17: 0.0 Giovani Bernard
x18: 0.0 Giovani Bernard /King
x19: 0.0 Jalen Reagor
x20: 0.0 Jalen Reagor /King
x21: 0.0 Cameron Brate
x22: 0.0 Cameron Brate /King
x23: 1.0 Quez Watkins
x24: 0.0 Quez Watkins /King
x25: 0.0 Jake Elliott
x26: 0.0 Jake Elliott /King
x27: 0.0 Ryan Succop
x28: 0.0 Ryan Succop /King
x29: 0.0 Zach Ertz
x30: 0.0 Zach Ertz /King
x31: 0.0 Kenneth Gainwell
x32: 0.0 Kenneth Gainwell /King
x33: 1.0 O.J. Howard
x34: 0.0 O.J. Howard /King
x35: 0.0 Tyler Johnson
x36: 0.0 Tyler Johnson /King
x37: 0.0 Greg Ward
x38: 0.0 Greg Ward /King


In [124]:
m.Equation(x1+x2+x9+x10+x11+x12+x13+x14==4)


m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model6 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           65
   Intermediates:            0
   Connections  :            0
   Equations    :           35
   Residuals    :           35
 
 Number of state variables:             65
 Number of total equations: -           34
 Number of slack variables: -           27
 ---------------------------------------
 Degrees of freedom       :              4
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    5 Dpth:    0 Lvs:    3 Obj: -9.36E+01 Gap:       NaN
--Integer Solution:  -9.23E+01 Lowes

In [125]:
m.Equation(x1+x2+x9+x10+x11+x12+x13+x14+x33+x34==5)


m.options.SOLVER = 1 # APOPT solver
m.solve()
print('x1: ' + str(x1.value[0])+ ' '+Download.iloc[0]['Name'])
print('x2: ' + str(x2.value[0])+ ' '+Download.iloc[0]['Name']+ ' /King')
print('x3: ' + str(x3.value[0])+ ' '+Download.iloc[1]['Name'])
print('x4: ' + str(x4.value[0])+ ' '+Download.iloc[1]['Name']+ ' /King')
print('x5: ' + str(x5.value[0])+ ' '+Download.iloc[2]['Name'])
print('x6: ' + str(x6.value[0])+ ' '+Download.iloc[2]['Name']+ ' /King')
print('x7: ' + str(x7.value[0])+ ' '+Download.iloc[3]['Name'])
print('x8: ' + str(x8.value[0])+ ' '+Download.iloc[3]['Name']+ ' /King')
print('x9: ' + str(x9.value[0])+ ' '+Download.iloc[4]['Name'])
print('x10: ' + str(x10.value[0])+ ' '+Download.iloc[4]['Name']+ ' /King')
print('x11: ' + str(x11.value[0])+ ' '+Download.iloc[5]['Name'])
print('x12: ' + str(x12.value[0])+ ' '+Download.iloc[5]['Name']+ ' /King')
print('x13: ' + str(x13.value[0])+ ' '+Download.iloc[6]['Name'])
print('x14: ' + str(x14.value[0])+ ' '+Download.iloc[6]['Name']+ ' /King')
print('x15: ' + str(x15.value[0])+ ' '+Download.iloc[7]['Name'])
print('x16: ' + str(x16.value[0])+ ' '+Download.iloc[7]['Name']+ ' /King')
print('x17: ' + str(x17.value[0])+ ' '+Download.iloc[8]['Name'])
print('x18: ' + str(x18.value[0])+ ' '+Download.iloc[8]['Name']+ ' /King')
print('x19: ' + str(x19.value[0])+ ' '+Download.iloc[9]['Name'])
print('x20: ' + str(x20.value[0])+ ' '+Download.iloc[9]['Name']+ ' /King')
print('x21: ' + str(x21.value[0])+ ' '+Download.iloc[10]['Name'])
print('x22: ' + str(x22.value[0])+ ' '+Download.iloc[10]['Name']+ ' /King')
print('x23: ' + str(x23.value[0])+ ' '+Download.iloc[11]['Name'])
print('x24: ' + str(x24.value[0])+ ' '+Download.iloc[11]['Name']+ ' /King')
print('x25: ' + str(x25.value[0])+ ' '+Download.iloc[12]['Name'])
print('x26: ' + str(x26.value[0])+ ' '+Download.iloc[12]['Name']+ ' /King')
print('x27: ' + str(x27.value[0])+ ' '+Download.iloc[13]['Name'])
print('x28: ' + str(x28.value[0])+ ' '+Download.iloc[13]['Name']+ ' /King')
print('x29: ' + str(x29.value[0])+ ' '+Download.iloc[14]['Name'])
print('x30: ' + str(x30.value[0])+ ' '+Download.iloc[14]['Name']+ ' /King')
print('x31: ' + str(x31.value[0])+ ' '+Download.iloc[15]['Name'])
print('x32: ' + str(x32.value[0])+ ' '+Download.iloc[15]['Name']+ ' /King')
print('x33: ' + str(x33.value[0])+ ' '+Download.iloc[16]['Name'])
print('x34: ' + str(x34.value[0])+ ' '+Download.iloc[16]['Name']+ ' /King')
print('x35: ' + str(x35.value[0])+ ' '+Download.iloc[17]['Name'])
print('x36: ' + str(x36.value[0])+ ' '+Download.iloc[17]['Name']+ ' /King')
print('x37: ' + str(x37.value[0])+ ' '+Download.iloc[18]['Name'])
print('x38: ' + str(x38.value[0])+ ' '+Download.iloc[18]['Name']+ ' /King')

apm 108.183.51.97_gk_model6 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           65
   Intermediates:            0
   Connections  :            0
   Equations    :           36
   Residuals    :           36
 
 Number of state variables:             65
 Number of total equations: -           35
 Number of slack variables: -           27
 ---------------------------------------
 Degrees of freedom       :              3
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    0 Obj: -9.12E+01 Gap:  0.00E+00
 Successful solution
 
 ------------